# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T10502'))

### GDPNow

In [6]:
url = ('https://api.stlouisfed.org/fred/series?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
update = r['seriess'][0]['last_updated']
url = ('https://api.stlouisfed.org/fred/series/observations?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
lv = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index()[-1]
lvt = f'{float(lv):.1f}'
nowdt = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index().index[-1]
ltdt = dtxt(nowdt)['qtr1']

s = ['A191RL']
gdp = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
gdpdt = gdp.index[-1]

In [7]:
mark = ' (see \\tikz \draw[blue!70!green, fill=cyan] (2.5pt,2.5pt) circle (2.5pt);)'
if dtxt(gdpdt)['datetime'] < nowdt:
    linedt = dtxt(pd.to_datetime(nowdt) - pd.DateOffset(days=46))['datetime']
    node = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{lvt}}}}}}}, '+
            'circle, draw=blue!70!green, fill=cyan, inner sep=1.8pt] at '+
            f'(axis cs:{nowdt}, {lv}) {{}};'+
            f'\draw [dashed] (axis cs:{{{linedt}}},\pgfkeysvalueof{{/pgfplots/ymin}})'+
            f' -- (axis cs:{{{linedt}}}, \pgfkeysvalueof{{/pgfplots/ymax}});')
    write_txt(text_dir / 'gdpnow_node.txt', node)

    node = ('\\node[circle, draw=blue!70!green, fill=cyan, inner sep=1.0pt] '+
            f'at (axis cs:{nowdt}, {lv}) {{}};')
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    # Nowcast text
    write_txt(text_dir / 'nowcast_text.txt', 'nowcast')
    latest_txt = 'latest '

else:
    node = ''
    write_txt(text_dir / 'gdpnow_node.txt', node)
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    write_txt(text_dir / 'nowcast_text.txt', '\color{white}{.}')
    latest_txt = ''
    mark = ''
            
text = (f'The {latest_txt}nowcast for {ltdt} is {lvt} percent, as '+
        f'of {dtxt(pd.to_datetime(update))["day1"]}{mark}.')
write_txt(text_dir / 'gdpnow.txt', text)
print(text)

The latest nowcast for 2022 Q4 is 4.4 percent, as of November 16, 2022 (see \tikz \draw[blue!70!green, fill=cyan] (2.5pt,2.5pt) circle (2.5pt);).


### GDP components

In [5]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

srs = {'A191RX': '\hspace{0.5mm} {\color{red!95!black}\\textbf{---}} Gross Domestic Product', 
       'DPCERX': '\hspace{2.5mm} {\color{yellow!45!orange}\\textbf{---}} Consumer Spending', 
       'A006RX': '\hspace{2.5mm} {\color{blue!70!black}\\textbf{---}} Gross Private Domestic Investment', 
       'A822RX': '\hspace{2.5mm} {\color{cyan!60!white}\\textbf{---}} Government Spending and Investment',
       'A019RX': '\hspace{2.5mm} {\color{green!60!black}\\textbf{---}} Net Exports', 
       'A020RX': '\hspace{7.5mm} Exports', 'A021RX': '\hspace{7.5mm} Less: Imports'}
s = [key for key, value in srs.items()]
s2 = [i[:-1].replace('A02', 'B02') + 'C' for i in s]
df = nipa_df(retrieve_table('T10106')['Data'], s).sort_index()
df2 = nipa_df(retrieve_table('T10105')['Data'], s2).sort_index()

dfpop = df.div(pop, axis=0)
real_vals = df2.rename({i: i.replace('RC', 'RX').replace('B02', 'A02') 
                        for i in s2}, axis=1).iloc[-1]
data = ((dfpop / dfpop.iloc[-1]) * (real_vals / pop.iloc[-1])).loc['1989':]

sel_cols = ['DPCERX', 'A006RX', 'A019RX', 'A822RX']

data[sel_cols].to_csv(data_dir / 'gdp_pc_comp.csv', index_label='date')

pce = f'\${data.DPCERX.iloc[-1] * 1000:,.0f}'
inv = f'\${data.A006RX.iloc[-1] * 1000:,.0f}'
gov = f'\${data.A822RX.iloc[-1] * 1000:,.0f}'
exp = f'\${abs(data.A019RX.iloc[-1]) * 1000:,.0f}'
ltdt = dtxt(data.index[-1])['qtr1']
pce_diff = f'\${(data.DPCERX.iloc[-1] - data.DPCERX.iloc[0]) * 1000:,.0f}'

text = ('Much of the increase in real GDP per capita over the past '+
        '30 years comes from consumer spending. Domestic consumer '+
        'spending (see {\color{yellow!45!orange}\\textbf{---}}) is '+
        f'equivalent to {pce} per person in {ltdt}, a price-adjusted '+
        f'increase of {pce_diff} since 1989. Gross private domestic '+
        'investment (see {\color{blue!70!black}\\textbf{---}}) is '+
        f'equivalent to {inv} per person in {ltdt}, and government '+
        'spending and investment (see {\color{cyan!60!white}\\textbf{---}}) '+
        f'totals {gov} per person. Net exports equivalent to {exp} per '+
        'person are subtracted to reflect only domestic production '+
        '(see {\color{green!60!black}\\textbf{---}}).')

write_txt(text_dir / 'gdp_pc_comp.txt', text)
write_txt(text_dir / 'gdp_ltdt.txt', ltdt)
print(text)

res = data * 1000
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).iloc[0]
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdppc_levels.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

Much of the increase in real GDP per capita over the past 30 years comes from consumer spending. Domestic consumer spending (see {\color{yellow!45!orange}\textbf{---}}) is equivalent to \$52,516 per person in 2022 Q3, a price-adjusted increase of \$23,159 since 1989. Gross private domestic investment (see {\color{blue!70!black}\textbf{---}}) is equivalent to \$13,775 per person in 2022 Q3, and government spending and investment (see {\color{cyan!60!white}\textbf{---}}) totals \$13,426 per person. Net exports equivalent to \$2,705 per person are subtracted to reflect only domestic production (see {\color{green!60!black}\textbf{---}}).


/tmp/ipykernel_22596/584066455.py:57: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  table.to_csv(data_dir / 'gdppc_levels.tex', sep='&',


### Labor Share

In [6]:
s = ['A261RC', 'A4002C', 'A262RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).sort_index()
df['Share'] = (df['A4002C'] / (df['A261RC'] - df['A262RC'])) * 100
data = df.loc['1989':, 'Share'].dropna()
data.to_csv(data_dir / 'laborshare.csv', 
            index_label='date', 
            float_format='%g')

node = end_node(data, 'blue!60!cyan', date='q')
write_txt(text_dir / 'laborshare_node.txt', node)

s = series_info(data)
ltdate = dtxt(s['date_latest'])['qtr2']
one_yr = value_text(s['val_latest'] - s['val_year_ago'], ptype='pp')
ltmin = s['val_latest'] - s['val_min']
ltmax = s['val_max'] - s['val_latest']
text = (f'As of {ltdate}, labor receives {s["val_latest"]:.1f} '+
        f"percent of net domestic income. Labor's share {one_yr} "+
        f'over the past year. The labor share is {ltmin:.1f} '+
        f'percentage points above its 30-year low of {s["val_min"]:.1f} '+
        f'percent in {s["date_min_ft"]}, and {ltmax:.1f} percentage '+
        f'points below the 30-year high of {s["val_max"]:.1f} percent '+
        f'in {s["date_max_ft"]}. ')
write_txt(text_dir / 'laborshare.txt', text)
print(text)

As of the second quarter of 2022, labor receives 63.2 percent of net domestic income. Labor's share decreased 0.9 percentage point over the past year. The labor share is 1.9 percentage points above its 30-year low of 61.3 percent in 2014 Q3, and 4.5 percentage points below the 30-year high of 67.7 percent in 2020 Q2. 


### GDP growth rate

In [8]:
s = ['A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')
date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'
write_txt(data_dir / 'gdp.txt', txt)

In [9]:
dfa = (nipa_df(retrieve_table('T10101A')['Data'], ['A191RL'])
       ['A191RL'])
dfa.index = dfa.index + pd.DateOffset(months=6)
dfa.to_csv(data_dir / 'gdp_a.csv', index_label='date')

In [10]:
df = nipa_df(retrieve_table('T10502')['Data'],
             ['A191RL'])['A191RL']
# Dates for latest quarters chart
a = (df.iloc[-4:].to_frame()
   .join(pd.Series({i: dtxt(i)['qtr4'].replace(' Q', '\\\Q')
                    for i in df.index[-4:]}, 
                   name='Quarter')))
next_qtr = a.index[-1] + pd.DateOffset(months=3)
dtxt(next_qtr)['qtr4'].replace(' Q', '\\\Q')
frow = pd.DataFrame({'Quarter': dtxt(next_qtr)['qtr4'].replace(' Q', '\\\Q'), 
                     'index': next_qtr}, index={'index': 4})
# From GDP Now (cover cases where no nowcast available)
if dtxt(gdpdt)['datetime'] < nowdt:
    a = pd.concat([a.reset_index(), frow]).set_index('index')
    a.loc[next_qtr, 'A191RL'] = 0
a['zero'] = 0
a.to_csv(data_dir/'gdp_rec.csv', index_label='date')
# Text
lty = df.index[-1].year
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
pry = df.index[-2].year
pr2y = df.index[-3].year
prdt = dtxt(df.index[-2])
prdt = prdt['qtr3'] if lty == pry else prdt['qtr1']
pr2dt = dtxt(df.index[-3])
pr2dt = pr2dt['qtr3'] if lty == pr2y else pr2dt['qtr1']
gtot = df.loc['1989':].mean()
g9300 = df.loc['1993':'2000'].mean()
g0013 = df.loc['2001':'2013'].mean()
g1419 = df.loc['2014':'2019'].mean()
g20on = df.loc['2020':].mean()
cov1 = value_text(df.loc['2020-04-01'])
cov2 = value_text(df.loc['2020-07-01'], 'increase_by')
ltval = value_text(df.iloc[-1], adj='annual')
prval = value_text(df.iloc[-2], 'increase_of')
prval2 = value_text(df.iloc[-3], 'increase_of')

color='red'
url = 'https://www.bea.gov/data/gdp/gross-domestic-product'
text = ('Real GDP growth \href{url}{measures} changes in '+
        'economic activity. As seen in the previous subsection, '+
        'real GDP has increased steadily over the long-term. '+
        'Since 1989, growth averaged '+
        f'{gtot:.1f} percent per year {c_box(color)}. '+
        'Growth rates were relatively high during the mid- '+
        f'to late-1990s, averaging {g9300:.1f} percent '+
        'from 1993 to 2000.\n\nIn the 2000s, the housing '+
        'bubble boosted GDP but then collapsed, '+
        f'leading to average growth of only {g0013:.1f} '+
        'percent from 2001 to 2013. Growth was slightly '+
        f'stronger from 2014 to 2019, averaging {g1419:.1f} '+
        'percent per year.\n\nIn 2020, COVID-19 caused '+
        'an economic shutdown, followed by monetary '+
        'and fiscal stimulus, resulting in large swings in '+
        f'GDP. Real GDP {cov1} in Q2, and {cov2} in Q3, by '+
        'far the largest changes in recent history. Since 2020, '+
        f'economic growth averaged {g20on:.1f} '+
        'percent.')
write_txt(text_dir / 'gdp_gr.txt', text)
print(text)

txt2 = ('The bottom-left chart shows annual growth, to make '+
        'trends more visible. The bottom-right chart shows '+
        'the most-recent four quarters and the estimate for the '+
        'current quarter. In the \\textbf{latest data}, '+
        f'covering {ltdt}, real GDP {ltval}, compared to {prval} '+
        f'in {prdt}, and {prval2} in {pr2dt}.')
write_txt(text_dir / 'gdp_gr2.txt', txt2)
print('\n', txt2)

Real GDP growth \href{url}{measures} changes in economic activity. As seen in the previous subsection, real GDP has increased steadily over the long-term. Since 1989, growth averaged 2.5 percent per year (see \cbox{red}). Growth rates were relatively high during the mid- to late-1990s, averaging 3.8 percent from 1993 to 2000.

In the 2000s, the housing bubble boosted GDP but then collapsed, leading to average growth of only 1.8 percent from 2001 to 2013. Growth was slightly stronger from 2014 to 2019, averaging 2.4 percent per year.

In 2020, COVID-19 caused an economic shutdown, followed by monetary and fiscal stimulus, resulting in large swings in GDP. Real GDP decreased 29.9 percent in Q2, and increased by 35.3 percent in Q3, by far the largest changes in recent history. Since 2020, economic growth averaged 2.6 percent.

 The bottom-left chart shows annual growth, to make trends more visible. The bottom-right chart shows the most-recent four quarters and the estimate for the current

### Recessions Table

In [4]:
rec = fred_df('USREC')
rec.to_csv(data_dir / 'recessions_raw.csv', index_label='date')

In [5]:
rec = pd.read_csv(data_dir / 'recessions_raw.csv', 
                  index_col='date', parse_dates=True)
first = rec[(rec.VALUE==1) & (rec.VALUE.shift(1) == 0)]
post = rec[(rec.VALUE==0) & (rec.VALUE.shift(1) == 1)]
names = [' \ Early `90s Recession', ' \ Early `00s Recession', 
         ' \ Great Recession', ' \ COVID-19 Recession']
recs = (pd.Series(data=first.index, index=names)
        .rename('First').to_frame())
recs['Last'] = rec[(rec.VALUE==1) & (rec.VALUE.shift(-1) == 0)].index
recs['Pre'] = rec[(rec.VALUE==0) & (rec.VALUE.shift(-1) == 1)].index
recs['Post'] = post.index
dur = [i.n for i in (post.index.to_period('M') - 
                     first.index.to_period('M'))]
recs['Dur'] = pd.Series(data=dur, index=recs.index)
recs['PrevEnd'] = recs['Post'].shift(1)
recs.loc[' \ Early `90s Recession', 'PrevEnd'] = pd.to_datetime('1989-01-01')
recs['NextStart'] = recs['Pre'].shift(-1)
recs.loc[' \ COVID-19 Recession', 'NextStart'] = cps_date()
recs['Start'] = recs.First.apply(lambda x: dtxt(x)['mon2'])
recs['End'] = recs.Last.apply(lambda x: dtxt(x)['mon2'])
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])['A191RX']
unrate = pd.read_csv(data_dir / 'jobs_report_main.csv', index_col='date', 
                 parse_dates=True)['Total']
for row in recs.itertuples():
    # Real GDP change
    vprev = rgdp.loc[:row.Pre].max()
    vmin = rgdp.loc[row.First:row.NextStart].min()
    ch = ((vmin / vprev) - 1) * 100
    recs.loc[row.Index, 'GDPch'] = ch
    # Unemployment rate change and duration
    pravg = unrate.loc[row.Pre - pd.DateOffset(years=3): row.Pre].mean()
    vmax = unrate.loc[row.First:row.NextStart].max()
    uch = vmax - pravg
    rdt = (unrate.loc[row.Last:].loc[(unrate <= pravg)].index[0] 
           if pravg >= unrate.iloc[-1] else '--')
    rtime = (int((rdt.to_period('M') - row.Last.to_period('M')).n) 
             if rdt != '--' else '--')
    recs.loc[row.Index, 'Unratech'] = uch    
    recs.loc[row.Index, 'RecoDate'] = rdt
    recs.loc[row.Index, 'RecoTime'] = str(rtime)
recs['GDPcht'] =  recs.GDPch.apply('{:.1f}'.format)
recs['Uncht'] =  recs.Unratech.apply('+{:.1f}'.format)
tbl = recs[['Start', 'End', 'Dur', 'GDPcht', 'Uncht', 'RecoTime']]
tbl.columns = ['Start \ \ \ Month', 'End \ \ \ \ \ \ Month', 
               'Recession Duration, Months', 
               'GDP Percent Change', 'Unemp. Rate Change*', 
               'Unemp. Rate Recovery, Months**']
tbl.to_csv(data_dir / 'recession.tex', sep='&', 
           lineterminator='\\\ ', quotechar=' ')

un3 = unrate.rolling(3).mean()
sahm = (un3 - un3.rolling(12).min()).dropna()
sahm.to_csv(data_dir / 'sahm.csv', index_label='date', 
            header=True)
bar = pd.Series(index=[sahm.index[0], sahm.index[-1]], 
                data=[0.5, 0.5], name='Bar')
bar.to_csv(data_dir / 'sahm_bar.csv', index_label='date', 
           header=True)
node = end_node(bar, 'gray', loc='start')
write_txt(text_dir / 'sahm_bar_node.txt', node)
marks = (sahm.loc[(sahm > 0.5) & (sahm.shift(1) < 0.5)]
             .rename('Mark').to_frame())
marks['Intersect'] = len(marks) * [0.5]
marks.to_csv(data_dir / 'sahm_marks.csv', index_label='date')

dur90 = numbers[f'{recs.Dur.iloc[0]:.1f}']
unrec90 = recs.RecoTime.iloc[0]
unrec00 = round(int(recs.RecoTime.iloc[1]) / 12)
durgr = recs.Dur.iloc[2]
unrecgr = recs.RecoTime.iloc[2]
durco = numbers[f'{recs.Dur.iloc[3]:.1f}']
gdpco = abs(recs.GDPch.iloc[3])

text = ('During the early 1990s recession, output contracted '+
        f'for {dur90} months and unemployment was higher '+
        f'than its pre-recession average for {unrec90} months. '+
        'The drop in output was smaller during the '+
        'early 2000s recession, but unemployment rates '+
        f'took almost {unrec00} years to recover.\n\n'+
        'The 2008--2009 great recession, caused by the '+
        'collapse of a housing bubble, was very severe. '+
        f'The recession lasted {durgr} months, with higher '+
        f'rates of unemployment lasting {unrecgr} months. The '+
        'most-recent COVID-19 recession was extremely severe '+
        f'and also extremely short-lived, lasting only {durco} '+
        f'months, but with output reduced {gdpco:.1f} percent.')
write_txt(text_dir / 'recessions.txt', text)
print(text)

During the early 1990s recession, output contracted for eight months and unemployment was higher than its pre-recession average for 63 months. The drop in output was smaller during the early 2000s recession, but unemployment rates took almost 16 years to recover.

The 2008--2009 great recession, caused by the collapse of a housing bubble, was very severe. The recession lasted 18 months, with higher rates of unemployment lasting 89 months. The most-recent COVID-19 recession was extremely severe and also extremely short-lived, lasting only two months, but with output reduced 9.6 percent.


### GDP Intro Chart and Text

In [12]:
df = nipa_df(retrieve_table('T70100')['Data'], ['A939RC', 'A939RX'])
df['value'] = (df['A939RX'] / df['A939RX'].iloc[-1]) * df['A939RC'].iloc[-1]
df.loc['1989':, 'value'].divide(1000).to_csv(data_dir / 'gdppc.csv', 
                                             index_label='date')
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
rgdp_cd = rgdp / rgdp.iloc[-1] * cd

ltdate = dtxt(df.index[-1])['qtr2']
gdp_val = int(rgdp_cd.iloc[-1].values[0] / 1000)
gdp_prv_val = int(rgdp_cd.loc['2019-10-01'] / 1000)
gdp_prv_val2 = int(rgdp_cd.iloc[0].values[0] / 1000)
gdppc_val = int(df.iloc[-1].loc['value'])
gdppc_prv_val = int(df.loc['2019-10-01', 'value'])
gdppc_prv_val2 = int(df.iloc[0].loc['value'])
qdate = dtxt(df.index[-1])['qtr1']
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']
num = numbers[f'{(pop.pct_change(4).iloc[-1] * 1000).round()}']
popnum = f'{num}-tenths'
color = 'red!95!black'
cl = c_line(color)
node = end_node(df['value'] / 1000, color, anchor='south', 
                dollar='thousands')
write_txt(text_dir / 'gdp_pc_node.txt', node)
text = (f'\${gdp_val:,} billion in {ltdate}, compared to an '+
        f'inflation-adjusted equivalent of \${gdp_prv_val:,} '+
        f'billion in 2019 Q4, and \${gdp_prv_val2:,} billion in '+
        'the first quarter of 1989.\n\nThe US population is growing '+
        f'by about {popnum} of a percent per year. GDP per capita '+
        f'{cl}, adjusted for inflation to {qdate} dollars, had '+
        f'increased to \${gdppc_prv_val:,} in 2019 Q4 from '+
        f'\${gdppc_prv_val2:,} in 1989 Q1, and is currently '+
        f'\${gdppc_val:,}, as of {ltdate}.')
write_txt(text_dir / 'gdp1.txt', text)
print(text)

\$25,663 billion in the third quarter of 2022, compared to an inflation-adjusted equivalent of \$24,630 billion in 2019 Q4, and \$11,190 billion in the first quarter of 1989.

The US population is growing by about three-tenths of a percent per year. GDP per capita (see {\color{red!95!black}\textbf{---}}), adjusted for inflation to 2022 Q3 dollars, had increased to \$74,364 in 2019 Q4 from \$45,824 in 1989 Q1, and is currently \$77,011, as of the third quarter of 2022.


### Private fixed investment

In [13]:
# In levels and share of GDP
s = ['A191RC', 'A007RC', 'A008RC', 'A011RC']
df = nipa_df(retrieve_table('T10105')['Data'], s) / 1_000_000
lt = df.iloc[-1]
sh = lt / lt.A191RC * 100
ldate = dtxt(df.index[-1])['qtr2']

# contribution to growth
s2 = ['A008RY', 'A011RY', 'A014RY']
df2 = nipa_df(retrieve_table('T10502')['Data'], s2)
df2.loc['1989':].to_csv(data_dir / 'inv.csv', index_label='date')
lt2 = df2.iloc[-1]

pfi = lt2[['A008RY', 'A011RY']].sum()
tot = value_text(pfi, 'contribution_to', ptype='pp', 
                 digits=2, threshold=0.01)
txt = lambda x: value_text(x, 'contribution', ptype='pp', 
                           digits=2, threshold=0.01)
dft = lt2.apply(txt)

text = (f'In {ldate}, private fixed investment, which does not '+
        f'include inventory investment, totals \${lt.A007RC:.1f} '+
        f'trillion, equivalent to {sh.A007RC:.1f} percent of '+
        f'GDP. Non-residential (business) fixed investment '+
        f'totals \${lt.A008RC:.1f} trillion, or {sh.A008RC:.1f} '+
        f'percent of GDP, while residential fixed investment '+
        f'totals \${lt.A011RC:,.1f} trillion ({sh.A011RC:.1f} '+
        'percent of GDP).\n\nDuring the quarter, private fixed '+
        f'investment {tot} real GDP growth. Non-residential fixed '+
        f'investment {dft.A008RY}, while residential fixed '+
        f'investment {dft.A011RY}. The change in private '+
        f'inventories {dft.A014RY}.')
write_txt(text_dir / 'inv_text.txt', text)
print(text)

In the third quarter of 2022, private fixed investment, which does not include inventory investment, totals \$4.5 trillion, equivalent to 17.5 percent of GDP. Non-residential (business) fixed investment totals \$3.4 trillion, or 13.2 percent of GDP, while residential fixed investment totals \$1.1 trillion (4.3 percent of GDP).

During the quarter, private fixed investment subtracted 0.88 percentage point from real GDP growth. Non-residential fixed investment contributed 0.49 percentage point, while residential fixed investment subtracted 1.37 percentage points. The change in private inventories subtracted 0.70 percentage point.


### Imports and Exports

In [14]:
# Import and Export share of GDP
s = ['B020RC', 'B021RC', 'B648RC', 'LA000006']
s2 = ['A191RC']
df = nipa_df(retrieve_table('T40205')['Data'], s)
df['A191RC'] = nipa_df(retrieve_table('T10105')['Data'], s2)
df['EX'] = df['B020RC'] - df['LA000006']
df['IM'] = df['B021RC'] - df['B648RC']
data = (df.div(df['A191RC'], axis=0) * 100).loc['1989':]
data.to_csv(data_dir / 'eximgdp.csv', index_label='date')

In [15]:
excol = 'green!60!teal!80!black'
imcol = 'blue!90!cyan'

node = end_node(data.IM, imcol, date='q',
                percent=True, offset=-0.1, full_year=True)
write_txt(text_dir / 'np_im_node.txt', node)
node = end_node(data.EX, excol, percent=True)
write_txt(text_dir / 'np_ex_node.txt', node)

date = dtxt(data.index[-1])['qtr2']
pc_dt = '2019-10-01'
pc_date = dtxt(pd.to_datetime(pc_dt))['qtr1']
text = ('Nonpetroleum goods and services imports '+
        f'{c_line(imcol)} were equivalent to '+
        f'{data.IM.iloc[-1]:.1f} percent of GDP '+
        f'in the {date}, while exports of nonpetroleum '+
        f'goods and services {c_line(excol)} were '+
        f'equivalent to {data.EX.iloc[-1]:.1f} percent '+
        f'of GDP. In {pc_date}, nonpetroleum imports '+
        f'were {data.IM.loc[pc_dt]:.1f} percent of GDP, '+
        f'and exports were {data.EX.loc[pc_dt]:.1f} percent.')
write_txt(text_dir / 'exim.txt', text)
print(text)

Nonpetroleum goods and services imports (see {\color{blue!90!cyan}\textbf{---}}) were equivalent to 14.2 percent of GDP in the the third quarter of 2022, while exports of nonpetroleum goods and services (see {\color{green!60!teal!80!black}\textbf{---}}) were equivalent to 10.6 percent of GDP. In 2019 Q4, nonpetroleum imports were 13.1 percent of GDP, and exports were 10.7 percent.


### Goods Import Penetration

In [16]:
s = ['A353RC']

G = nipa_df(retrieve_table('T10205')['Data'], s).sort_index()

s = ['A253RC', 'A255RC', 'B647RC', 'LA000004', 'A650RC', 
     'B651RC', 'A652RC', 'A653RC', 'B648RC']

MX = nipa_df(retrieve_table('T40205')['Data'], s).sort_index()

D = G['A353RC'] - MX['A253RC'] + MX['A255RC']
result = (MX['A255RC'] / D)

import_categories = ['B647RC', 'LA000004', 'A650RC', 'B651RC', 
                     'A652RC', 'A653RC', 'B648RC']
Msh = MX[import_categories].div(MX['A255RC'], axis=0)

Msh['Consumer'] = Msh['B647RC'] + Msh['A652RC'] + Msh['B651RC']
Msh['Capital'] = Msh['LA000004'] - Msh['B648RC'] + Msh['A650RC'] + Msh['A653RC']

final = Msh[['Consumer', 'Capital', 'B648RC']].multiply(result, axis=0) * 100

final.loc['1989':].to_csv(data_dir / 'goodsimpsh.csv', index_label='date')

ltdate = final.index[-1]
ltdt = dtxt(ltdate)['qtr1']
cons, capi = [value_text(final[i].iloc[-1], style='eq') 
              for i in ['Consumer', 'Capital']]
oil = final['B648RC'].iloc[-1]
text = (f'As of {ltdt}, imports of consumer goods excluding petroleum '+
        f'and petroleum products are {cons} of domestic consumption of '+
        'goods (see\cbox{cyan!40!white}). Petroleum-related imports claim '+
        f'{oil:.1f} percent (see\cbox{{purple}}) and imports of all other '+
        f'goods, primarily capital goods, industrial supplies, and materials, '+
        f'are {capi} (see\cbox{{blue!50!cyan}}).')
write_txt(text_dir / 'goodsimpsh1.txt', text)
print(text, '\n')

ch11 = (final.loc['2011-01-01'] - final.iloc[0])
cons11, pet11, oth11 = [value_text(ch11[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
chlt = (final.iloc[-1] - final.loc['2011-01-01'])
conslt, petlt, othlt = [value_text(chlt[i], adj='equivalent', threshold=0.1) 
                        for i in ['Consumer', 'B648RC', 'Capital']]
text = ('From 1989 to 2011, imports of consumer goods excluding '+
        f'petroleum {cons11} of domestic consumption of goods; petroleum-'+
        f'related imports {pet11}; and all other goods imports {oth11}. \n\n'+
        f'Since 2011, imports of consumer goods {conslt} of domestic goods '+
        f'demand; imports of petroleum products {petlt}; and other '+
        f'imports {othlt}.')
write_txt(text_dir / 'goodsimpsh.txt', text)
print(text)

As of 2022 Q3, imports of consumer goods excluding petroleum and petroleum products are equivalent to 15.5 percent of domestic consumption of goods (see\cbox{cyan!40!white}). Petroleum-related imports claim 3.4 percent (see\cbox{purple}) and imports of all other goods, primarily capital goods, industrial supplies, and materials, are equivalent to 16.2 percent (see\cbox{blue!50!cyan}). 

From 1989 to 2011, imports of consumer goods excluding petroleum increased by the equivalent of 5.7 percent of domestic consumption of goods; petroleum-related imports increased by the equivalent of 6.3 percent; and all other goods imports increased by the equivalent of 6.2 percent. 

Since 2011, imports of consumer goods decreased by the equivalent of 0.9 percent of domestic goods demand; imports of petroleum products decreased by the equivalent of 4.9 percent; and other imports was virtually unchanged.


### ITA - Financial Account Balance

In [17]:
ind_list = ['FinAssetsExclFinDeriv', 'FinLiabsExclFinDeriv', 
            'FinDeriv', 'StatDisc']

api_results = bea_api_ita(ind_list, bea_key)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

df = pd.DataFrame(
    {name: {i['TimePeriod']: i['DataValue'] 
            for i in json.loads(series)['BEAAPI']['Results']['Data']} 
     for name, series in api_results})
df.index = pd.to_datetime(df.index)
df = (df.replace(r'^\s*$', np.nan, regex=True)
        .astype('float').rolling(4).sum())
df['FAB'] = df['FinLiabsExclFinDeriv'] - df['FinAssetsExclFinDeriv']
df['TOT'] = df[['FAB', 'StatDisc', 'FinDeriv']].sum(axis=1)
final = (df.divide(gdp, axis=0).dropna(how='all') * 100).fillna(0)
keep_cols = ['FinDeriv', 'StatDisc', 'FAB', 'TOT']
final.loc['1989':, keep_cols].to_csv(data_dir/'fab.csv', index_label='date')

s = final.iloc[-1]
liab = value_text(s.FinLiabsExclFinDeriv, style='eq', adj='GDP')
assets = value_text(s.FinAssetsExclFinDeriv, style='eq', adj='GDP')
ldate = dtxt(final.index[-1])['qtr1']
text = (f'Over the year ending {ldate}, net domestic acquisitions of '+
        f'foreign assets were {assets}, while net domestic incurrence '+
        f'of foreign liabilities was {liab}. As a result, domestic '+
        f'net borrowing totals {s.TOT:.1f} percent of GDP over the '+
         'one-year period.')
write_txt(text_dir / 'fab.txt', text)
print(text)

Over the year ending 2022 Q2, net domestic acquisitions of foreign assets were equivalent to 5.2 percent of GDP, while net domestic incurrence of foreign liabilities was equivalent to 8.2 percent of GDP. As a result, domestic net borrowing totals 3.4 percent of GDP over the one-year period.


### GDP Composition

In [18]:
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY']
df = nipa_df(retrieve_table('T10502')['Data'], s).loc['1989':]
df.to_csv(data_dir / 'comp.csv', index_label='date')

ltdt = dtxt(df.index[-1])['qtr2']
sl = [('DPCERY', 'contribution_to'), ('A006RY', 'contribution_to'), 
      ('A822RY', 'contribution'), ('A019RY', 'contribution')]
d = {s: value_text(df[s].iloc[-1], style=style, ptype='pp', 
                   digits=2, threshold=0.1) for s, style in sl}
text = (f'In {ltdt}, consumer spending (see\cbox{{yellow!80!orange}}) '+
        f'{d["DPCERY"]} overall real GDP growth. Private domestic '+
        f'investment (see\cbox{{blue!70!black}}) {d["A006RY"]} real GDP '+
        'growth, government spending and investment (see\cbox{cyan!50!white}) '+
        f'{d["A822RY"]}, and net exports (see\cbox{{green!60!black}}) '+
        f'{d["A019RY"]}.')  
write_txt(text_dir / 'gdp_exp_basic.txt', text)
print(text)

In the third quarter of 2022, consumer spending (see\cbox{yellow!80!orange}) contributed 0.97 percentage point to overall real GDP growth. Private domestic investment (see\cbox{blue!70!black}) subtracted 1.59 percentage points from real GDP growth, government spending and investment (see\cbox{cyan!50!white}) contributed 0.42 percentage point, and net exports (see\cbox{green!60!black}) contributed 2.77 percentage points.


### Gross Domestic Income

In [19]:
s = ['A261RX', 'W256RX']
rgdi = nipa_df(retrieve_table('T11706')['Data'], s).dropna()

s = ['A261RC', 'A4002C', 'W056RC', 'A107RC', 'W271RC', 'A262RC',
     'A4102C', 'A038RC']

df = nipa_df(retrieve_table('T11000')['Data'], s).dropna()

pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

# Calculate indirect taxes net of transfers
df['indirect'] = df['W056RC'] - df['A107RC']

# Calculate GDI deflator from real GDI series
deflator = rgdi['A261RX'] / df['A261RC']
deflator = deflator / deflator.iloc[-1]
df = df.multiply(deflator, axis=0)

# Calculate contributions to growth
dft = df.diff()
dft = dft.div(dft['A261RC'], axis=0)
contr = dft.multiply((((df['A261RC'].pct_change() + 1) ** 4) - 1) * 100, axis=0)

cols = ['A261RC', 'W271RC', 'A4002C', 'A262RC', 'indirect']

contr.loc['1989':, cols].to_csv(data_dir / 'gdi.csv', index_label='date')

dfpop = df.div(pop, axis=0).dropna()

ltdt = dtxt(dfpop.index[-1])['qtr1']
write_txt(text_dir / 'gdi_ltdt.txt', ltdt)

dfpop[cols].to_csv(data_dir / 'gdi_pc_comp.csv', index_label='date')

In [20]:
srs = {'A261RC': 'Gross Domestic Income', 
       'A4002C': '\hspace{0.1mm} {\color{magenta!90!blue}\\textbf{---}} Labor', 
       'A4102C': '\hspace{6mm} Wages and Salaries',
       'A038RC': '\hspace{6mm} Supplements',
       'W271RC': '\hspace{0.1mm} {\color{yellow!60!orange}\\textbf{---}} Profit', 
       'indirect': '\hspace{0.1mm} {\color{violet}\\textbf{---}} Indirect Taxes', 
       'W056RC': '\hspace{6mm} Taxes on Production and Imports',
       'A107RC': '\hspace{6mm} Less: Subsidies',
       'A262RC': '\hspace{0.1mm} {\color{teal!60!white}\\textbf{---}} Depreciation'}

res = (dfpop[list(srs.keys())] * 1000).dropna()
lt = res.rename(srs, axis=1).iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.rename(srs, axis=1).loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.rename(srs, axis=1).loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.rename(srs, axis=1).loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'gdipc_levels.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')
print(table)

prdate = '2019-10-01'
ltval = df['A261RC'].iloc[-1] / 1000
ltpc = dfpop['A261RC'].iloc[-1] * 1000
prval = df['A261RC'].loc[prdate] / 1000
prpc = dfpop['A261RC'].loc[prdate] * 1000
prdt = dtxt(pd.to_datetime(prdate))['qtr1']
text = (f'of \${ltval:,.0f} billion in {ltdt}, compared to an inflation-'+
        f'adjusted equivalent of \${prval:,.0f} billion in {prdt}. Real '+
        f'GDI per capita was \${ltpc:,.0f} in {ltdt} and \${prpc:,.0f} '+
        f'in {prdt}.')
write_txt(text_dir / 'gdi_levels.txt', text)
print('\n', text, '\n')

l_pc = dfpop['A4002C'].iloc[-1] * 1000
l_pr = dfpop['A4002C'].loc[prdate] * 1000
k_pc = dfpop['W271RC'].iloc[-1] * 1000
k_pr = dfpop['W271RC'].loc[prdate] * 1000
t_pc = dfpop['indirect'].iloc[-1] * 1000
t_pr = dfpop['indirect'].loc[prdate] * 1000
d_pc = dfpop['A262RC'].iloc[-1] * 1000
d_pr = dfpop['A262RC'].loc[prdate] * 1000

text = ('Gross labor income per capita is equivalent '+
        f'to \${l_pc:,.0f} in {ltdt} '+
        '(see {\color{magenta!90!blue}\\textbf{---}}) '+
        f'and \${l_pr:,.0f} in 2019 Q4, on an annualized, '+
        'seasonally-adjusted, and inflation-adjusted basis. '+
        f'Profits per person total \${k_pc:,.0f} in {ltdt} '+
        '(see {\color{yellow!60!orange}\\textbf{---}}) and '+
        f'\${k_pr:,.0f} in {prdt}, following the same adjustments. '+
        f'Indirect taxes less subsidies per capita total \${t_pc:,.0f} '+
        f'in {ltdt} (see {{\color{{violet}}\\textbf{{---}}}}) '+
        f'and \${t_pr:,.0f} in {prdt}. Lastly, depreciation per '+
        f'capita is \${d_pc:,.0f} in {ltdt} (see '+
        '{\color{teal!60!white}\\textbf{---}}) and '+
        f'\${d_pr:,.0f} in {prdt}.')
write_txt(text_dir / 'gdi_levels_pc.txt', text)
print(text)

                                                     2022 Q2 2019 Q4 2000 Q1  \
Gross Domestic Income                               \$76,805  73,984  59,258   
\hspace{0.1mm} {\color{magenta!90!blue}\textbf{...    40,496  39,449  33,584   
\hspace{6mm} Wages and Salaries                       33,406  32,179  27,763   
\hspace{6mm} Supplements                               7,090   7,270   5,821   
\hspace{0.1mm} {\color{yellow!60!orange}\textbf...    18,606  17,734  13,232   
\hspace{0.1mm} {\color{violet}\textbf{---}} Ind...     4,962   4,976   3,813   
\hspace{6mm} Taxes on Production and Imports           5,333   5,256   4,076   
\hspace{6mm} Less: Subsidies                             371     280     263   
\hspace{0.1mm} {\color{teal!60!white}\textbf{--...    12,741  11,825   8,629   

                                                   1989 Q1  
Gross Domestic Income                               46,745  
\hspace{0.1mm} {\color{magenta!90!blue}\textbf{...  26,209  
\hspace{6mm} Wag

/tmp/ipykernel_22596/1225961163.py:22: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  table.to_csv(data_dir / 'gdipc_levels.tex', sep='&',


In [21]:
s = ['A261RC', 'A4002C', 'W271RC', 'indirect', 'A262RC']
df = contr[s]
gdi_lt = inc_dec_percent(df["A261RC"].iloc[-1], annualized=True)
gdi_pr = inc_dec_percent(df["A261RC"].iloc[-2], how='of')
gdi_pr2 = inc_dec_percent(df["A261RC"].iloc[-3], how='of')

ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
prdt2 = dtxt(df.index[-3])['qtr1']

sl = [('A4002C', 'main'), ('W271RC', 'main'), 
      ('indirect', 'end'), ('A262RC', 'end')]

d = {}
for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)
    
text1 = (f'In {ltdt}, gross domestic income {gdi_lt}, following {gdi_pr} '+
         f'in {prdt} and {gdi_pr2} in {prdt2}. ')  

l_lt = cont_subt(df['A4002C'].iloc[-1])
l_pr = cont_subt(df['A4002C'].iloc[-2], style='of')
k_lt = cont_subt(df['W271RC'].iloc[-1]).replace(' from', '').replace(' to', '')
k_pr = cont_subt(df['W271RC'].iloc[-2]).replace(' from', '').replace(' to', '')
if k_lt[:3] == k_pr[:3]:
    if abs(df['W271RC'].iloc[-2]) > 1:
        pp = 'percentage points'
    else:
        pp = 'percentage point'
    k_pr = f"{abs(df['W271RC'].iloc[-2]):.2f} {pp}"
t_lt = cont_subt(df['indirect'].iloc[-1])
t_pr = cont_subt(df['indirect'].iloc[-2]).replace(' from', '').replace(' to', '')
    
text = (f'{text1}In the latest quarter, labor income {l_lt} overall growth, '+
        f'following a {l_pr} in {prdt}. Profit income {k_lt} in '+
        f'{ltdt} and {k_pr} in {prdt}. Changes in indirect tax revenue '+
        f'and surpluses {t_lt} aggregate income growth in the latest quarter and '+
        f'{t_pr} in {prdt}. ')
write_txt(text_dir / 'gdi_growth_comp.txt', text)
print(text) 

cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
cont_subt will be removed from a future version
In the second quarter of 2022, gross domestic income was virtually unchanged, following an increase of 0.8 percent in 2022 Q1 and an increase of 6.7 percent in 2021 Q4. In the latest quarter, labor income subtracted 1.30 percentage points from overall growth, following a subtraction of 0.75 percentage point in 2022 Q1. Profit income contributed 0.57 percentage point in the second quarter of 2022 and subtracted 1.68 percentage points in 2022 Q1. Changes in indirect tax revenue and surpluses contributed 0.18 percenta

### Consumer Spending Overview (Levels)

In [45]:
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']

n = {'DPCERC': 'Total',
     'DGDSRC': 'Goods',
     'DSERRC': 'Services',
     'DHUTRC': 'Housing',
     'A011RC': 'ResInv',
     'DMOTRC': 'MotorVeh',
     'DFDHRC': 'Furn',
     'DREQRC': 'RecDG',
     'DFXARC': 'Groc',
     'DCLORC': 'Cloth',
     'DHLCRC': 'Health',
     'DTRSRC': 'Transp',
     'DRCARC': 'RecSer',
     'DFSARC': 'FoodAcc',
     'DIFSRC': 'FinIns'}
n2 = {k[:-2] + 'RA': v for k, v in n.items()}
s = list(n.keys())
s2 = list(n2.keys())

othserv = lambda x: x['Services'] - x['Housing']
shelter = lambda x: x['Housing'] + x['ResInv']

df = (nipa_df(retrieve_table('T10505')['Data'], s)
      .rename(n, axis=1))
df2 = (nipa_df(retrieve_table('T10503')['Data'], s2)
       .rename(n2, axis=1))
real = ((df2 / df2.iloc[-1]) * df.iloc[-1]).assign(OTHSERV = othserv, SHELTER = shelter)
pp = real.divide(pop, axis=0)

keep_col = ['Goods', 'Services', 'Housing', 'ResInv', 'OTHSERV', 'SHELTER']
pp.loc['1989':, keep_col].to_csv(data_dir / 'pce_levels.csv', index_label='date')

lttot = real.Total.iloc[-1] / 1_000_000
prtot = real.Total.iloc[-2] / 1_000_000
pctot = real.Total.loc['2019-10-01'] / 1_000_000
ltdate = dtxt(real.index[-1])['qtr1']
prdate = dtxt(real.index[-2])['qtr1']
totpp = pp.Total.iloc[-1] * 1_000
goodpp = pp.Goods.iloc[-1] * 1_000
servpp = pp.Services.iloc[-1] * 1_000
goodpppc = pp.Goods.loc['2019-10-01'] * 1_000
servpppc = pp.Services.loc['2019-10-01'] * 1_000

text = ('Total consumer spending is '+
        f'\${lttot:.1f} trillion in {ltdate}, compared to a price-adjusted '+
        f'\${prtot:.1f} trillion in {prdate} and \${pctot:.1f} trillion in 2019 Q4. '+
        'On a per person basis, consumer spending is '+
        f'\${totpp:,.0f} in {ltdate}, of which \${goodpp:,.0f} are spent on goods '+
        '(see {\color{red}\\textbf{---}}) and '+
        f'\${servpp:,.0f} on services '+
        '(see {\color{orange}\\textbf{---}}). In the fourth quarter of 2019, '+
        f'before the pandemic, consumer spending on goods was \${goodpppc:,.0f} '+
        f'per person, and spending on services was \${servpppc:,.0f} per person, '+
        'after adjusting for inflation. ')
write_txt(text_dir / 'pce_levels.txt', text)
print(text)

hult = pp['Housing'].iloc[-1] * 1_000
hupc = pp['Housing'].loc['2019-10-01'] * 1_000
rfilt = pp['ResInv'].iloc[-1] * 1_000
rfipc = pp['ResInv'].loc['2019-10-01'] * 1_000

text = ('Within consumer spending on services, housing and utilities spending '+
        f'totals \${hult:,.0f} on an annualized and per person basis in {ltdate} '+
        '(see {\color{green!60!blue}\\textbf{---}}) '+
        f'and \${hupc:,.0f} in 2019 Q4. Construction or improvement '+
        'of housing is considered residential fixed investment, not '+
        'consumer spending, but can be combined with spending to analyze '+
        'patterns in shelter costs. In '+
        f'{ltdate}, residential investment totals \${rfilt:,.0f} per person '+
        '(see {\color{blue!80!black}\\textbf{---}}), '+
        f'compared to \${rfipc:,.0f} in the pre-COVID data covering 2019 Q4. ')
write_txt(text_dir / 'pce2_levels.txt', text)
print('\n', text)

othlt = pp['OTHSERV'].iloc[-1] * 1_000
othpr = pp['OTHSERV'].iloc[-2] * 1_000
othpc = pp.loc['2019-10-01', 'OTHSERV'] * 1_000
chval = ((othlt / othpc) - 1) * 100
chtxt = value_text(chval)
shellt = pp['SHELTER'].iloc[-1] * 1_000
shelpr = pp['SHELTER'].iloc[-2] * 1_000
shelpc = pp.loc['2019-10-01', 'SHELTER'] * 1_000
shelmax = pp['SHELTER'].max() * 1_000
shelmaxdt = dtxt(pp.SHELTER.idxmax())['qtr2']

text = ('Consumer spending on services other than housing and utilities totals \$'+
        f'{othlt:,.0f} per person, on an annualized basis, in {ltdate} '+
        '(see {\color{blue!75!white}\\textbf{---}}), '+
        f'compared to \${othpr:,.0f} in {prdate}, and \${othpc:,.0f} in '+
        f'2019 Q4. Spending on non-housing services has {chtxt} since 2019 Q4. '+
        '\n\nShelter costs, which combine housing, utilities, and residential fixed '+
        f'investment, are \${shellt:,.0f} per person in {ltdate} '+
        '(see {\color{green!85!blue}\\textbf{---}})'+
        f', \${shelpr:,.0f} '+
        f'in {prdate}, and \${shelpc:,.0f} in 2019 Q4. Shelter spending peaked at '+
        f'\${shelmax:,.0f} per person in {shelmaxdt}, during the housing bubble.')
write_txt(text_dir / 'pce3_levels.txt', text)
print('\n', text)

Total consumer spending is \$17.5 trillion in 2022 Q3, compared to a price-adjusted \$17.4 trillion in 2022 Q2 and \$16.4 trillion in 2019 Q4. On a per person basis, consumer spending is \$52,516 in 2022 Q3, of which \$17,935 are spent on goods (see {\color{red}\textbf{---}}) and \$34,581 on services (see {\color{orange}\textbf{---}}). In the fourth quarter of 2019, before the pandemic, consumer spending on goods was \$15,662 per person, and spending on services was \$33,665 per person, after adjusting for inflation. 

 Within consumer spending on services, housing and utilities spending totals \$9,097 on an annualized and per person basis in 2022 Q3 (see {\color{green!60!blue}\textbf{---}}) and \$8,902 in 2019 Q4. Construction or improvement of housing is considered residential fixed investment, not consumer spending, but can be combined with spending to analyze patterns in shelter costs. In 2022 Q3, residential investment totals \$3,324 per person (see {\color{blue!80!black}\textbf{-

In [46]:
n = {'Total': 'Total',
     'Goods': '\hspace{0.1mm} {\color{red}\\textbf{---}} Goods',
     'MotorVeh': '\hspace{5mm} Motor Vehicles and Parts',
     'Furn': '\hspace{5mm} Furniture and HH Equipment',
     'RecDG': '\hspace{5mm} Recreational Durable Goods',
     'Groc': '\hspace{5mm} Groceries',
     'Cloth': '\hspace{5mm} Clothes and Shoes',
     'OTHSERV': '\hspace{0.1mm} {\color{blue!75!white}\\textbf{---}} Services ex. Shelter',
     'Health': '\hspace{5mm} Health Care Services',
     'Transp': '\hspace{5mm} Transportation',
     'RecSer': '\hspace{5mm} Recreational',
     'FoodAcc': '\hspace{5mm} Food and Accommodations',
     'FinIns': '\hspace{5mm} Financial and Insurance',
     'SHELTER': '\hspace{0.1mm} {\color{green!85!blue}\\textbf{---}} Shelter ',
     'Housing': '\hspace{5mm} Housing Services and Utilities ',
     'ResInv': '\hspace{5mm} Residential Fixed Investment'}

res = pp[n.keys()].rename(n, axis=1) * 1_000
lt = res.iloc[-1]
lt.name = dtxt(lt.name)['qtr1']
pr = res.loc['2019-10-01']
pr.name = dtxt(pr.name)['qtr1']
p00 = res.loc['2000-01-01']
p00.name = dtxt(p00.name)['qtr1']
fi = res.loc['1989-01-01']
fi.name = dtxt(fi.name)['qtr1']
table = pd.concat([lt, pr, p00, fi], axis=1).applymap('{:,.0f}'.format)
table.iloc[0, 0] = f'\${table.iloc[0, 0]}'
table.to_csv(data_dir / 'pce_levels.tex', sep='&', lineterminator='\\\ ', quotechar=' ')
table

,2022 Q3,2019 Q4,2000 Q1,1989 Q1
Total,"\$52,516","49,389","37,574","29,356"
\hspace{0.1mm} {\color{red}\textbf{---}} Goods,"17,935","15,662","10,493","7,873"
\hspace{5mm} Motor Vehicles and Parts,"2,144","2,059","1,787","1,345"
\hspace{5mm} Furniture and HH Equipment,"1,601","1,322",629,436
\hspace{5mm} Recreational Durable Goods,"1,960","1,289",270,79
\hspace{5mm} Groceries,"3,862","3,766","3,214","3,192"
\hspace{5mm} Clothes and Shoes,"1,493","1,251",930,690
\hspace{0.1mm} {\color{blue!75!white}\textbf{---}} Services ex. Shelter,"25,484","24,763","19,221","14,759"
\hspace{5mm} Health Care Services,"8,176","8,150","5,386","4,877"
\hspace{5mm} Transportation,"1,709","1,780","1,563","1,085"


### Consumer Spending and Residential Fixed Investment

In [24]:
s = ['DNPIRY', 'DSERRY', 'DPCERY', 'DGDSRY', 'DMOTRY',
     'DFDHRY', 'DREQRY', 'DFXARY', 'DCLORY', 'DHLCRY',
     'DTRSRY', 'DRCARY', 'DFSARY', 'DIFSRY', 'DHUTRY',
     'A011RY']

n = {'TOTAL': '& Total',
     'DGDSRY': '\cbox{red} & Goods',
     'DMOTRY': '& \hspace{1mm} Motor Vehicles and Parts',
     'DFDHRY': '& \hspace{1mm} Furniture and HH Equipment',
     'DREQRY': '& \hspace{1mm} Recreational Durable Goods',
     'DFXARY': '& \hspace{1mm} Groceries',
     'DCLORY': '& \hspace{1mm} Clothes and Shoes',
     'OTHSERV': '\cbox{blue!75!white} & Services (ex. Shelter)',
     'DHLCRY': '& \hspace{1mm} Health Care Services',
     'DTRSRY': '& \hspace{1mm} Transportation',
     'DRCARY': '& \hspace{1mm} Recreational',
     'DFSARY': '& \hspace{1mm} Food and Accommodations',
     'DIFSRY': '& \hspace{1mm} Financial and Insurance',
     'SHELTER': '\cbox{green!85!blue} & Shelter ',
     'DHUTRY': '& \hspace{1mm} Housing Services and Utilities ',
     'A011RY': '& \hspace{1mm} Residential Fixed Investment'}

total = lambda x: x['DPCERY']
othserv = lambda x: x['DSERRY'] - x['DHUTRY']
shelter = lambda x: x['DHUTRY'] + x['A011RY']

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      .assign(TOTAL = total, OTHSERV = othserv, SHELTER = shelter)
      [list(n.keys())])

#(df.loc['1989':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
#   .to_csv(data_dir / 'pce.csv', index_label='date', float_format='%g'))

(df.loc['1988':, ['DGDSRY', 'OTHSERV', 'SHELTER']]
 .rolling(4).mean().dropna()
 .to_csv(data_dir / 'pce_ma.csv', index_label='date', float_format='%g'))

data = df.iloc[-5:].iloc[::-1].T

cols = [f'& {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = df.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = df.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = df.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'pce.tex', sep='&', line_terminator='\\\ ', quotechar=' '))

ld = dtxt(df.index[-1])['qtr1']
prd = dtxt(df.index[-2])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']

totlt = df['TOTAL'].iloc[-1]
totltt = value_text(totlt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
totpr = df['TOTAL'].iloc[-2]
totprt = value_text(totpr, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)
totpc = df.loc['2019-10-01', 'TOTAL']
totpct = value_text(totpc, style='contribution_of', ptype='pp', 
                    digits=1, casual=True, threshold=0.1)

txt1 = (f'These categories {totltt} GDP growth in {ld} and {totprt} in '+
        f'{prd}, compared to {totpct} in 2019 Q4, before the pandemic.')
write_txt(text_dir / 'pce1.txt', txt1)
print(txt1)

gdslt = df['DGDSRY'].iloc[-1]
gdsltt = value_text(gdslt, style='contribution_to', ptype='pp', 
                    digits=1, threshold=0.1)
serlt = df['OTHSERV'].iloc[-1]
serltt = value_text(serlt, style='contribution', ptype='pp', digits=1, 
                    casual=True, threshold=0.1)
shelt = df['SHELTER'].iloc[-1]
sheltt = value_text(shelt, style='contribution', ptype='pp', 
                    digits=1, threshold=0.1)

txt2 = (f'In the {ld2}, household spending on goods {gdsltt} GDP growth, '+
         'household spending on services other than housing and utilities '+
        f'{serltt}, and shelter spending and investment {sheltt}.')
write_txt(text_dir / 'pce2.txt', txt2)
print('\n', txt2)

These categories contributed one percentage point to GDP growth in 2022 Q3 and contributed 1.4 percentage points in 2022 Q2, compared to an addition of 1.6 percentage points in 2019 Q4, before the pandemic.

 In the the third quarter of 2022, household spending on goods subtracted 0.3 percentage point from GDP growth, household spending on services other than housing and utilities added 1.2 percentage points, and shelter spending and investment subtracted 1.3 percentage points.


/tmp/ipykernel_22596/1007738796.py:49: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  (data.round(2).applymap('{:,.2f}'.format)


### Trade Contribution to GDP

In [11]:
s = ['A019RY', 'A253RY', 'A646RY', 'A255RY', 'A656RY']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'nx.csv', index_label='date')

sl = [('A253RY', 'contribution_to'), ('A646RY', 'contribution'), 
      ('A255RY', 'contribution_to'), ('A656RY', 'contribution')]

d = {}
for s, style in sl:
    d[s] = value_text(df[s].iloc[-1], style, 'pp', digits=2)
    
ldate = dtxt(df.index[-1])['qtr2']

text = (f"Goods exports {d['A253RY']} GDP growth in {ldate} while "+
        f"services exports {d['A646RY']}. Good imports {d['A255RY']} "+
        f"GDP growth and services imports {d['A656RY']}.")
write_txt(text_dir / 'trade.txt', text)
print(text)

Goods exports contributed 1.34 percentage points to GDP growth in the third quarter of 2022 while services exports contributed 0.29 percentage point. Good imports contributed 1.20 percentage points to GDP growth and services imports subtracted 0.06 percentage point.


### Business investment

In [26]:
s = ['Y001RY', 'A009RY', 'Y033RY']

df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'businv.csv', index_label='date')

In [27]:
s = ['Y001RC', 'B009RC', 'Y033RC', 'A008RC']
data = nipa_df(retrieve_table('T10105')['Data'], s)
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df1 = data.divide(gdp, axis=0) * 100
(df1.loc['1989':, ['Y001RC', 'B009RC', 'Y033RC']]
    .to_csv(data_dir / 'businvsh.csv', index_label='date'))

ltdate = dtxt(df1.index[-1])['qtr1']

levels = {}
shgdp = {}
for series in ['Y001RC', 'B009RC', 'Y033RC']:
    level_val = data[series].iloc[-1] / 1_000
    txt = f"\${level_val:,.0f} billion"
    levels[series] = txt
    shgdp[series] = f"{df1[series].iloc[-1]:.1f} percent of GDP"

text = ('Business investments in fixed assets are grouped into three '+
        'categories: structures, equipment, and intellectual property '+
        '(for example software and R\&D). Investment in structures was '+
        f'{levels["B009RC"]} in {ltdate}, equivalent to {shgdp["B009RC"]} '+
        '(see {\color{yellow!50!orange}\\textbf{---}}). '+
        f'Equipment investment was {levels["Y033RC"]} or {shgdp["Y033RC"]} '+
        '(see {\color{cyan!60!white}\\textbf{---}}), '+
        'and intellectual property investment '+
        f'was {levels["Y001RC"]} or {shgdp["Y001RC"]} '+
        '(see {\color{violet}\\textbf{---}}). ')
write_txt(text_dir / 'businv_sh.txt', text)
print(text)

Business investments in fixed assets are grouped into three categories: structures, equipment, and intellectual property (for example software and R\&D). Investment in structures was \$634 billion in 2022 Q3, equivalent to 2.5 percent of GDP (see {\color{yellow!50!orange}\textbf{---}}). Equipment investment was \$1,352 billion or 5.3 percent of GDP (see {\color{cyan!60!white}\textbf{---}}), and intellectual property investment was \$1,397 billion or 5.4 percent of GDP (see {\color{violet}\textbf{---}}). 


In [28]:
s = ['W790RC', 'W276RC', 'W987RC']
df = nipa_df(retrieve_table('T50100')['Data'], s).join(data['A008RC'])
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = (df.div(gdp, axis=0) * 100).dropna()
res.loc['1989':].to_csv(data_dir / 'businv_main.csv', index_label='date')

ltdate1 = dtxt(df.index[-1])['qtr1']
ltdate2 = dtxt(df.index[-1])['qtr2']

levels = {}
shgdp = {}
for series in ['W790RC', 'W276RC', 'W987RC', 'A008RC']:
    level_val = df[series].iloc[-1] / 1_000
    n = ''
    if level_val < 0:
        n = 'negative '
        level_val = abs(level_val)
    txt = f"{n}\${level_val:,.0f} billion"
    levels[series] = txt
    shgdp[series] = f"{res[series].iloc[-1]:.1f} percent of GDP"
    
diff = ((df['W987RC'].iloc[-1] / df.loc['2019-10-01', 'W987RC']) - 1) * 100
gdt = value_text(diff)
diff2 = ((df['W790RC'].iloc[-1] / df.loc['2019-10-01', 'W790RC']) - 1) * 100
if diff2 < -95:
    ndt = 'but collapsed completely'
else:
    ndt = 'and ' + value_text(diff2)
gpcl = f"{n}\${df.loc['2019-10-01', 'W987RC'] / 1_000:,.0f} billion"
npcl = f"{n}\${df.loc['2019-10-01', 'W790RC'] / 1_000:,.0f} billion"
if diff < 0:
    txt = 'as gross investment fell while deprecation was relatively constant'
else:
    txt = 'as gross investment recovered from its pandemic lows'
    
text = (f'In the {ltdate2}, gross private business investment totals '+
        f'{levels["W987RC"]} on a seasonally-adjusted annualized basis, '+
        f'equivalent to {shgdp["W987RC"]} '+
        '(see {\color{blue!60!violet}\\textbf{---}}). Private business investment '+
        f'in fixed assets totals {levels["A008RC"]}, or {shgdp["A008RC"]} '+
        '(see {\color{cyan!80!white}\\textbf{---}}). Private business depreciation '+
        f'totals {levels["W276RC"]} in the quarter, or {shgdp["W276RC"]} '+
        '(see {\color{magenta}\\textbf{---}}). '+
        f'As a result, net investment is {levels["W790RC"]}, or {shgdp["W790RC"]} '+
        '(see {\color{green!80!blue}\\textbf{---}}). \n\n '+
        'In 2019 Q4, prior to the COVID-19 pandemic, private business gross '+
        f'investment was {gpcl}. Since 2019 Q4, annualized gross investment {gdt}. '+
        f'Net investment was {npcl} in 2019 Q4, {ndt} from 2019 Q4 to {ltdate1}, '+
        f'{txt}. ')
write_txt(text_dir / 'businv_levels.txt', text)
print(text)  

In the the third quarter of 2022, gross private business investment totals \$3,440 billion on a seasonally-adjusted annualized basis, equivalent to 13.4 percent of GDP (see {\color{blue!60!violet}\textbf{---}}). Private business investment in fixed assets totals \$3,382 billion, or 13.2 percent of GDP (see {\color{cyan!80!white}\textbf{---}}). Private business depreciation totals \$2,819 billion in the quarter, or 11.0 percent of GDP (see {\color{magenta}\textbf{---}}). As a result, net investment is \$621 billion, or 2.4 percent of GDP (see {\color{green!80!blue}\textbf{---}}). 

 In 2019 Q4, prior to the COVID-19 pandemic, private business gross investment was \$2,880 billion. Since 2019 Q4, annualized gross investment increased 19.5 percent. Net investment was \$596 billion in 2019 Q4, and increased 4.1 percent from 2019 Q4 to 2022 Q3, as gross investment recovered from its pandemic lows. 


### Durable goods new orders

In [4]:
# New orders for capital goods excluding defense or aircraft
url = ('https://api.census.gov/data/timeseries/eits/advm3?'+
       f'get=cell_value,time_slot_id&key={census_key}&'+
       'category_code=NXA&data_type_code=NO&time=from+1992&'+
       'for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
        .set_index('date')['cell_value'].astype('float'))
df.to_csv(data_dir / 'dgno_raw.csv', index_label='date', 
          header=True)

In [5]:
df = pd.read_csv(data_dir / 'dgno_raw.csv', index_col='date', 
                 parse_dates=True)['cell_value']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
res = ((df.resample('QS').sum() * 4  / 
        gdp['A191RC']).dropna() * 100).iloc[1:]
(res.rename('value').to_csv(data_dir / 'dgno.csv', 
                            index_label='date',  header=True))
color = 'purple!50!violet'
node = end_node(res, color, date='q', percent=True)
write_txt(text_dir / 'dgno_node.txt', node)
ltval = f'\${df.iloc[-1] / 1000:,.0f} billion'
ldate = dtxt(df.index[-1])['mon1']
comp = pd.to_datetime('2020-02-01')
compdt = dtxt(pd.to_datetime(comp))['mon1']
val = value_text(df.pct_change(12).iloc[-1] * 100)
pcv = ((df.iloc[-1] / df.loc[comp]) - 1) * 100
pcval = value_text(pcv, 'increase_by')
text = ('New orders for manufactured core capital goods excluding '+
        f'aircraft totaled {ltval} in {ldate}, equivalent to '+
        f'{res.iloc[-1]:.1f} percent of GDP {c_line(color)}. New '+
        f'orders {val} over the past year, and {pcval} since '+
        f'{compdt}.')
write_txt(text_dir / 'dgno.txt', text)
print(text)

New orders for manufactured core capital goods excluding aircraft totaled \$75 billion in September 2022, equivalent to 3.5 percent of GDP (see {\color{purple!50!violet}\textbf{---}}). New orders increased 7.9 percent over the past year, and increased by 22.1 percent since February 2020.


### Government spending and investment

In [31]:
n = {'A822RY': 'Total',
     'A823RY': '\hspace{1mm}Federal total',
     'A824RY': '\hspace{1mm}\cbox{blue!60!black}National defense',
     'A997RY': '\hspace{7mm}Consumption expenditures',
     'A788RY': '\hspace{7mm}Gross investment',
     'A825RY': '\hspace{1mm}\cbox{green!85!black}Nondefense',
     'A542RY': '\hspace{7mm}Consumption expenditures',
     'A798RY': '\hspace{7mm}Gross investment',
     'A829RY': '\hspace{-2mm}\cbox{purple!70!magenta}State \& local total',
     'A991RY': '\hspace{5mm}Consumption expenditures',
     'A799RY': '\hspace{5mm}Gross investment'}

df = nipa_df(retrieve_table('T10502')['Data'], 
             list(n.keys()) + ['A191RL'])
df.rolling(4).mean().loc['1989':].to_csv(data_dir / 'gov.csv', 
                                         index_label='date')

In [32]:
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
tot = df.A822RY
gdp = df['A191RL'].iloc[-1]
oneyr = value_text(tot.iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
v89 = tot.loc['1989':].mean()
totval = value_text(tot.iloc[-1], 'contribution_to', 'pp', 
                    threshold=0.01, digits=2)
defval = value_text(df['A824RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
defcb = c_box('blue!60!black')
fedval = value_text(df['A825RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
fedcb = c_box('green!85!black')
slgval = value_text(df['A829RY'].iloc[-4:].mean(), 'contribution', 'pp', 
                   threshold=0.01, digits=2)
slgcb = c_box('purple!70!magenta')
text = ('Government consumption and investment directly affect economic '+
        f'growth in the short-term. In {ltdt}, government consumption spending '+
        f'and investment {totval} the real GDP growth rate of {gdp:.1f} '+
        'percent. Over the latest four quarters, government consumption '+
        f'and investment {oneyr} to economic growth, on average. '+
        f'Since 1989, the average contribution has been {v89:.2f} '+
        f'percentage points.\n\nOver the four quarters ending {ltdt2}, '+
        f'by level of government, national defense {defval} {defcb}, '+
        f'other federal government {fedval} {fedcb}, and state and '+
        f'local government {slgval} {slgcb}.')
write_txt(text_dir / 'gov.txt', text)
print(text)

Government consumption and investment directly affect economic growth in the short-term. In the third quarter of 2022, government consumption spending and investment contributed 0.42 percentage point to the real GDP growth rate of 2.6 percent. Over the latest four quarters, government consumption and investment subtracted 0.11 percentage point to economic growth, on average. Since 1989, the average contribution has been 0.23 percentage points.

Over the four quarters ending 2022 Q3, by level of government, national defense subtracted 0.08 percentage point (see \cbox{blue!60!black}), other federal government subtracted 0.01 percentage point (see \cbox{green!85!black}), and state and local government subtracted 0.02 percentage point (see \cbox{purple!70!magenta}).


In [33]:
result = df[n.keys()]
data = result.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
data = data.applymap('{:.2f}'.format)
data.to_csv(data_dir / 'gov.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

/tmp/ipykernel_22596/1943181507.py:14: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  data.to_csv(data_dir / 'gov.tex', sep='&', line_terminator='\\\ ', quotechar=' ')


### Government Net Investment

In [34]:
cofc = nipa_df(retrieve_table('T11000')['Data'], ['A264RC'])['A264RC']
ginv = nipa_df(retrieve_table('T30100')['Data'], ['A782RC'])['A782RC']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = (((ginv - cofc) / gdp) * 100).loc['1989':]
data.name = 'Value'
data.to_csv(data_dir / 'govnetinv.csv', index_label='date')

color = 'red'
node = end_node(data, color, percent=True, date='q', digits=2, 
                full_year=True)
write_txt(text_dir / 'govnetinv_node.txt', node)

ltdt = dtxt(data.index[-1])['qtr1']
yrdt = dtxt(data.index[-5])['qtr1']
yr2dt = dtxt(data.index[-9])['qtr1']
gni = f'\${(ginv - cofc).iloc[-1] / 1_000:.1f} billion'
gnish = f'{data.iloc[-1]:.2f} percent'
gnishyr = f'{data.iloc[-5]:.2f} percent'
gnishyr2 = f'{data.iloc[-9]:.2f} percent'

text = ("Government gross investment, less depreciation, is the government's "+
        "net investment in the tangible assets that make the economy more "+
        "productive. Government investment includes infrastructure, "+
        "buildings, equipment, intellectual property, and other capital goods. "+
        f"In the latest data, covering {ltdt}, government net investment is "+
        f"{gni}. Government net investment is equivalent to {gnish} "+
        f"of GDP in {ltdt} {c_line(color)}, compared "+
        f"to {gnishyr} in {yrdt}, and {gnishyr2} in {yr2dt}. ")
write_txt(text_dir / f'govnetinv.txt', text)
print(text)

Government gross investment, less depreciation, is the government's net investment in the tangible assets that make the economy more productive. Government investment includes infrastructure, buildings, equipment, intellectual property, and other capital goods. In the latest data, covering 2022 Q3, government net investment is \$130.9 billion. Government net investment is equivalent to 0.51 percent of GDP in 2022 Q3 (see {\color{red}\textbf{---}}), compared to 0.67 percent in 2021 Q3, and 0.85 percent in 2020 Q3. 


### Government receipts and expenditures

In [35]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

d = {'slggdp': {'name': 'combined state and local government', 
                'table': 'T30300', 'series': 
                {'W024RC': 'SLG_EXP', 'W023RC': 'SLG_REC'}},
     'fedgdp': {'name': 'federal government', 
                'table': 'T30200', 'series':
                {'W005RC': 'FED_REC', 'W013RC': 'FED_EXP'}}}

data = pd.DataFrame()

ltdate = dtxt(gdp.index[-1])['qtr1']

for group, details in d.items():
    df = nipa_df(retrieve_table(details['table'])['Data'], details['series'].keys())
    (df.div(gdp, axis=0) * 100).loc['1989':].to_csv(data_dir / f'{group}.csv', 
                                                    index_label='date', float_format='%g')
    
    grp = group[:3].upper()
    df.columns = [details['series'][i] for i in df.columns]
    bal = df[f'{grp}_REC'] - df[f'{grp}_EXP']
    bal_lt = bal.dropna().iloc[-1]
    bal_lt_dt = dtxt(bal.dropna().index[-1])['qtr1']
    def_sur = 'deficit' if bal_lt < 0 else 'surplus'
    df[f'{grp}_BAL'] = bal
    
    for col in df.columns:
        data[col] = df[col]
        data[col+'_GDP'] = df[col].div(gdp, axis=0) * 100
        
    bal_gdp = data[f'{grp}_BAL_GDP'].dropna().iloc[-1]
    bal_txt = (f'In {bal_lt_dt}, the {d[group]["name"]} {def_sur} was '+
               f'\${abs(bal_lt) / 1000:,.0f} billion or {abs(bal_gdp):.1f} percent of GDP. ')
    
    exp_txt = (f'{d[group]["name"].capitalize()} expenditures total '+
               f'\${data[grp+"_EXP"].iloc[-1] / 1000000:.1f} trillion, '+
               f'or {data[grp+"_EXP_GDP"].iloc[-1]:.1f} percent of GDP, in {ltdate}. ')
    
    if pd.isna(df[[i for i in df.columns if i[4:] == 'REC'][0]].iloc[-1]) == True:
        rec_txt = (f'BEA has not yet released receipts data for {ltdate}, however, '+
                   f'in {bal_lt_dt}, {d[group]["name"]} receipts total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion, '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    else:
        rec_txt = ('Receipts for the same period total '+
                   f'\${data[grp+"_REC"].dropna().iloc[-1] / 1000000:.1f} trillion '+
                   f'or {data[grp+"_REC_GDP"].dropna().iloc[-1]:.1f} percent of GDP. ')
    
    text = exp_txt + rec_txt + bal_txt
    write_txt(text_dir / f'{group}.txt', text)
    print(text)

Combined state and local government expenditures total \$3.6 trillion, or 14.0 percent of GDP, in 2022 Q3. BEA has not yet released receipts data for 2022 Q3, however, in 2022 Q2, combined state and local government receipts total \$3.5 trillion, or 14.0 percent of GDP. In 2022 Q2, the combined state and local government deficit was \$38 billion or 0.2 percent of GDP. 
Federal government expenditures total \$6.0 trillion, or 23.6 percent of GDP, in 2022 Q3. BEA has not yet released receipts data for 2022 Q3, however, in 2022 Q2, federal government receipts total \$5.1 trillion, or 20.1 percent of GDP. In 2022 Q2, the federal government deficit was \$864 billion or 3.4 percent of GDP. 


### Public Debt by Holder

In [5]:
series = ['FDHBATN', 'GFDEBTN', 'FDHBFRBN', 'FDHBPIN', 'FDHBFIN']
start = '1988-01-01'
ftype = '&file_type=json'
base = 'https://api.stlouisfed.org/fred/series/observations?'

df = pd.DataFrame()

for srs in series:
    param = f'series_id={srs}&observation_start={start}&api_key={fred_key}'
    url = f'{base}{param}{ftype}'
    r = requests.get(url).json()['observations']
    data = pd.Series({pd.to_datetime(i['date']): (float(i['value']) / 1000.0) 
                      if srs in series[:2] 
                      else float(i['value']) for i in r})
    df[srs] = data

In [6]:
df = df.dropna()
df['PD'] = df['FDHBPIN'] - df['FDHBFIN']
df['IG'] = df['GFDEBTN'] - (df['FDHBFRBN'] + df['FDHBPIN'])

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']

data = df.loc['1989':].div(gdp / 1000.0, axis=0).dropna()

(data[['PD', 'FDHBFIN', 'FDHBFRBN', 'IG']] * 100).to_csv(data_dir / 'pubdebt.csv', 
                                                         index_label='date')

ld = pd.to_datetime(df.index[-1])
ldate = dtxt(ld)['qtr2']

sh = df.div(df['GFDEBTN'], axis=0).iloc[-1] * 100
lv = df.iloc[-1] / 1000
dl = data.iloc[-1] * 100
url = 'https://www.fiscal.treasury.gov/reports-statements/treasury-bulletin/current.html'
text = (f'In {ldate}, total \href{{{url}}}{{public debt}} was '+
        f'\${lv.GFDEBTN:.1f} trillion, equivalent to {dl.GFDEBTN:.1f} percent '+
        f'of GDP. Of this, \${lv.PD:.1f} trillion, or {sh.PD:.1f} percent of '+
        'the total, is held by private domestic investors (see\cbox{green!60!black}). '+
        f'An additional \${lv.FDHBFIN:.1f} trillion, or {sh.FDHBFIN:.1f} percent '+
        'of the total, is held by foreign investors (see\cbox{orange!70!white}). '+
        'The remainder is held by the Federal Reserve (see\cbox{blue}) '+
        'and various government agencies and trusts (see\cbox{cyan!50!white}), '+
        'such as the Social Security Trust Fund. ')
write_txt(text_dir / 'pubdebt.txt', text)
print(text)

In the second quarter of 2022, total \href{https://www.fiscal.treasury.gov/reports-statements/treasury-bulletin/current.html}{public debt} was \$30.6 trillion, equivalent to 121.1 percent of GDP. Of this, \$10.7 trillion, or 35.1 percent of the total, is held by private domestic investors (see\cbox{green!60!black}). An additional \$7.4 trillion, or 24.3 percent of the total, is held by foreign investors (see\cbox{orange!70!white}). The remainder is held by the Federal Reserve (see\cbox{blue}) and various government agencies and trusts (see\cbox{cyan!50!white}), such as the Social Security Trust Fund. 


### Personal Income

In [38]:
# deflator
d = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG']
deflator = d.iloc[-1] / d

# collect and combine series
s = ['A065RC', 'A033RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC', 'A061RC']
df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              TRANSFER = lambda x: x['A577RC'] - x['A061RC'])
      .drop(['A061RC', 'A041RC', 'A048RC', 'W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
growth_contrib(df, 'A065RC').loc['1989':].to_csv(data_dir / 'pi.csv', index_label='date')

data = growth_contrib(df, 'A065RC').rename({'A065RC': 'TOTAL', 'A033RC': 'LABOR'}, axis=1)

ltdate = dtxt(data.index[-1])['qtr1']
tot = value_text(data['TOTAL'].iloc[-1], adj='annualized', digits=2)
slist = ['LABOR', 'CAPITAL', 'TRANSFER']
style = {k: 'contribution' for k in slist}
style['LABOR'] = 'contribution_to'
d = {}
for i in slist:
    d[i] = value_text(data[i].iloc[-1], style=style[i], 
                      ptype='pp', digits=2)
text = (f'Aggregate real personal income {tot} in {ltdate}. '+
        f'Labor income {d["LABOR"]} overall growth, capital '+
        f'income {d["CAPITAL"]}, and welfare income {d["TRANSFER"]}. ')
write_txt(text_dir / 'pi.txt', text)
print(text)

Aggregate real personal income increased at an annualized rate of 1.26 percent in 2022 Q3. Labor income contributed 1.64 percentage points to overall growth, capital income contributed 0.58 percentage point, and welfare income subtracted 0.96 percentage point. 


### Government Personal Income

In [39]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['B230RC']

population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']

s = ['A063RC', 'B202RC', 'A061RC', 'W055RC']
df = nipa_df(retrieve_table('T20100')['Data'], s)

n = {'A063RC': 'Welfare',
     'B202RC': 'WandS'}
result = df.multiply(deflator, axis=0).divide(population, axis=0)
result['TaxSI'] = result['A061RC'] + result['W055RC']
result = result.drop(['A061RC', 'W055RC'], axis=1).rename(n, axis=1)
result.loc['1989':].to_csv(data_dir / 'govpi.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
wsltval = result['WandS'].iloc[-1] * 1_000
wsprval = result.loc[prdt, 'WandS'] * 1_000
wltval = result['Welfare'].iloc[-1] * 1_000
wprval = result.loc[prdt, 'Welfare'] * 1_000
w89val = result.loc['1989-01-01', 'Welfare'] * 1_000
tltval = result['TaxSI'].iloc[-1] * 1_000
tprval = result.loc[prdt, 'TaxSI'] * 1_000
t89val = result.loc['1989-01-01', 'TaxSI'] * 1_000

text = (f'In {ltdate}, government worker wages and salaries, not including '+
        f'benefits, were equivalent to \${wsltval:,.0f} per capita, following a '+
        f'price-adjusted \${wsprval:,.0f} in {prdate} '+
        '(see {\color{orange}\\textbf{---}}). Welfare payments were equivalent '+
        f'to \${wltval:,.0f} per capita in {ltdate}, compared to \${wprval:,.0f} '+
        f'per capita in {prdate} '+
        '(see {\color{violet}\\textbf{---}}). In 1989 Q1, welfare payments were '+
        f'equivalent to \${w89val:,.0f} per person.  \n\n Personal current taxes '+
        f'and social insurance contributions total \${tltval:,.0f} per capita '+
        f'in {ltdate}, \${tprval:,.0f} in {prdate}, and \${t89val:,.0f} in 1989 '+
        '(see {\color{cyan!80!blue}\\textbf{---}}).')
write_txt(text_dir / 'govpi.txt', text)
print(text)

In 2022 Q3, government worker wages and salaries, not including benefits, were equivalent to \$4,859 per capita, following a price-adjusted \$5,008 in 2019 Q4 (see {\color{orange}\textbf{---}}). Welfare payments were equivalent to \$11,440 per capita in 2022 Q3, compared to \$10,543 per capita in 2019 Q4 (see {\color{violet}\textbf{---}}). In 1989 Q1, welfare payments were equivalent to \$4,275 per person.  

 Personal current taxes and social insurance contributions total \$14,733 per capita in 2022 Q3, \$12,369 in 2019 Q4, and \$7,831 in 1989 (see {\color{cyan!80!blue}\textbf{---}}).


### Government Consumption and Investment

In [40]:
s = ['A823RC', 'A829RC', 'A191RC', 'A824RC', 'A825RC']
df = nipa_df(retrieve_table('T10105')['Data'], s).sort_index()
result = df.drop('A191RC', axis=1).divide(df['A191RC'], axis=0) * 100
result.loc['1989':].to_csv(data_dir / 'govci.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
prdt = pd.to_datetime('2019-10-01')
prdate = dtxt(prdt)['qtr1']
fndnom = df['A825RC'].iloc[-1] / 1_000
fndlt = result['A825RC'].iloc[-1]
fndpr = result.loc[prdt, 'A825RC']
dlt = result['A824RC'].iloc[-1]
dpr = result.loc[prdt, 'A824RC']
d89 = result.loc['1989-01-01', 'A824RC']
slt = result['A829RC'].iloc[-1]
spr = result.loc[prdt, 'A829RC']

text = (f'In {ltdate}, federal non-defense spending and investment '+
        f'was \${fndnom:,.1f} billion, equivalent to {fndlt:.1f} percent '+
        'of GDP (see {\color{green!85!black}\\textbf{---}}), compared '+
        f'to {fndpr:.1f} percent of GDP in {prdate}. Federal spending '+
        f'on national defense was equivalent to {dlt:.1f} percent of '+
        f'GDP in the latest quarter and {dpr:.1f} percent in {prdate} '+
        '(see {\color{blue!60!black}\\textbf{---}}). National defense '
        f'spending was {d89:.1f} percent of GDP in 1989 Q1. \n\n In '+
        f'{ltdate}, state and local government spending and investment '+
        f'was equivalent to {slt:.1f} percent of GDP, compared to '+
        f'{spr:.1f} percent in {prdate} '+
        '(see {\color{purple!70!magenta}\\textbf{---}}).')
write_txt(text_dir / 'govci.txt', text)
print(text)

In 2022 Q3, federal non-defense spending and investment was \$721.8 billion, equivalent to 2.8 percent of GDP (see {\color{green!85!black}\textbf{---}}), compared to 2.7 percent of GDP in 2019 Q4. Federal spending on national defense was equivalent to 3.6 percent of GDP in the latest quarter and 4.0 percent in 2019 Q4 (see {\color{blue!60!black}\textbf{---}}). National defense spending was 6.9 percent of GDP in 1989 Q1. 

 In 2022 Q3, state and local government spending and investment was equivalent to 11.0 percent of GDP, compared to 11.0 percent in 2019 Q4 (see {\color{purple!70!magenta}\textbf{---}}).


### Consumer Spending Growth Contributions

In [41]:
s = ['B230RC']
population = nipa_df(retrieve_table('T20100')['Data'], s)['B230RC']
s = ['DPCERG']
d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = ['A067RC', 'A068RC', 'A071RC', 'DPCERC']

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(OTHER = lambda x: -(x['A068RC'] - x['DPCERC']),
              SAVING = lambda x: -x['A071RC'])
      .drop(['A068RC'], axis=1)
      .divide(population, axis=0)
      .multiply(deflator, axis=0))

data = growth_contrib(df, 'DPCERC').rolling(4).mean()
data3y = growth_contrib(df, 'DPCERC').rolling(12).mean()
data.loc['1989':].to_csv(data_dir / 'pcedecomp.csv', index_label='date')

In [42]:
date = f'{data.index[-1].year} Q{data.index[-1].quarter}'
pcetext = value_text(data['DPCERC'].iloc[-1], adj='average')

slist = ['A067RC', 'SAVING', 'OTHER']
d = {}
for i in slist:
    val = data[i].iloc[-1]
    vt = value_text(data[i].iloc[-1], style='contribution', 
           ptype='pp', casual=True)
    if f'{abs(val):.1f}' == '0.0':
        vt = "didn't affect the total"
    d[i] = vt
pce19 = value_text(data.loc['2019', 'DPCERC'].mean(), adj='average')
dpi19 = value_text(data.loc['2019', 'A067RC'].mean(), style='contribution', 
           ptype='pp')
save19 = value_text(data.loc['2019', 'SAVING'].mean(), style='contribution', 
           ptype='pp')

pcetxt = (f'Real per capita consumer spending {pcetext} over the '+
          f'four quarters ending {date}. Changes to disposable income '+
          f'{d["A067RC"]}, changes to saving {d["SAVING"]}, and '+
          f'changes to other outlays {d["OTHER"]}. '+
          f'During 2019, real per capita consumer spending {pce19}. '+
          f'Increased income {dpi19}, and a slight increase in saving {save19}.')
write_txt(text_dir / 'pcedecomp.txt', pcetxt)
print(pcetxt)

Real per capita consumer spending increased at an average rate of 1.7 percent over the four quarters ending 2022 Q3. Changes to disposable income subtracted 4.8 percentage points, changes to saving added 6.7 percentage points, and changes to other outlays subtracted 0.2 percentage point. During 2019, real per capita consumer spending increased at an average rate of 1.5 percent. Increased income contributed 3.3 percentage points, and a slight increase in saving subtracted 1.7 percentage points.


### Sectoral Accounts

**NOTE:** Need to convert "deficit", "borrower" etc to parameters!

In [43]:
def sect_txt(val):
    lb = ('borrower', 'deficit') if val < 0 else ('lender', 'surplus')
    d = {'v': val,
         'vt': f'{val:.1f}',
         'avt': f'{abs(val):.1f}',
         'vg': f'the equivalent of {abs(val):.1f} percent of GDP',
         'lb': lb[0],
         'ds': lb[1]}
    return d

In [44]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
s = ['W162RC', 'W994RC', 'AD01RC', 'W995RC', 'W996RC', 'AD03RC']
sb = nipa_df(retrieve_table('T50100')['Data'], s)
df = (sb.div(gdp, axis=0) * 100).dropna()
res = df[['W995RC', 'W996RC', 'AD03RC']].loc['1989':]
res.to_csv(data_dir / 'sectbal2.csv', index_label='date')

data = pd.DataFrame()
data['PRIV'] = df['W994RC']
data['GOV'] = df['AD01RC']
data['ROW'] = -df['W162RC'] # Negative in raw data
data = data.dropna().loc['1989':]
data.to_csv(data_dir / 'sectbal.csv', index_label='date')

date = dtxt(data.index[-1])['qtr1']
pcdt = '2019-10-01'

priv = sect_txt(data.PRIV.iloc[-1])
privpr = sect_txt(data.loc['2019', 'PRIV'].mean())
row = sect_txt(data.ROW.iloc[-1])
rowpr = sect_txt(data.loc['2019', 'ROW'].mean())
gov = sect_txt(data.GOV.iloc[-1])
govpr = sect_txt(data.loc['2019', 'GOV'].mean())

compare = compare_text(priv['v'], privpr['v'], [0.4, 1.0, 3.5])

txt = (f"In {date}, the US private sector was a net {priv['lb']} "+
       f"(running a {priv['ds']}) of {priv['vg']}, {compare} the "+
       f"{privpr['avt']} percent surplus in 2019. The rest of the world "+
       f"was a net {row['lb']} to the US to {row['vg']} in {date}, compared "+
       f"to {rowpr['avt']} percent in 2019. Balancing these transactions, "+
       "the government (federal, state, and local combined) was a net "+
       f"{gov['lb']} (running a {gov['ds']}) of {gov['vg']} in {date}, "+
       f"compared to {govpr['avt']} percent in 2019. ")
write_txt(text_dir / 'sectbal.txt', txt)
print(txt, '\n')

date = dtxt(df.index[-1])['qtr1']
hh = sect_txt(df.W996RC.iloc[-1])
hhpr = sect_txt(df.loc['2019', 'W996RC'].mean())
pb = sect_txt(df.W995RC.iloc[-1])
pbpr = sect_txt(df.loc['2019', 'W995RC'].mean())   
text = ('Breaking out the two main categories in the private sector, households '+
       f'were net {hh["lb"]}s (ran a {hh["ds"]}) of {hh["vg"]} in {date} '+
        '(see\cbox{orange!90!yellow}), while private businesses--corporate '+
       f'and noncorporate--were net {pb["lb"]}s of {pb["vg"]} '+
       '(see\cbox{purple!50!red}). In 2019, households were net '+
       f'{hhpr["lb"]}s of {hhpr["avt"]} percent, and private businesses were net '+
       f'{pbpr["lb"]}s of {pbpr["avt"]} percent.') 
write_txt(text_dir / 'sectbal2.txt', text)
print(text)

In 2022 Q2, the US private sector was a net borrower (running a deficit) of the equivalent of 0.5 percent of GDP, far below the 4.5 percent surplus in 2019. The rest of the world was a net lender to the US to the equivalent of 4.1 percent of GDP in 2022 Q2, compared to 2.2 percent in 2019. Balancing these transactions, the government (federal, state, and local combined) was a net borrower (running a deficit) of the equivalent of 3.6 percent of GDP in 2022 Q2, compared to 6.7 percent in 2019.  

Breaking out the two main categories in the private sector, households were net lenders (ran a surplus) of the equivalent of 0.8 percent of GDP in 2022 Q2 (see\cbox{orange!90!yellow}), while private businesses--corporate and noncorporate--were net borrowers of the equivalent of 1.3 percent of GDP (see\cbox{purple!50!red}). In 2019, households were net lenders of 5.4 percent, and private businesses were net borrowers of 0.9 percent.


### Current Account Balance

In [45]:
s = ['A191RC']

gdp = nipa_df(retrieve_table('T10105')['Data'], s)

n = {'A1073C': 'Current receipts',
     'B020RC': '\hspace{1mm}Exports',
     'A253RC': '\hspace{3mm}Goods',
     'A332RC': '\hspace{5mm}Durable',
     'A339RC': '\hspace{5mm}Nondurable',
     'A646RC': '\hspace{3mm}Services',
     'B645RC': '\hspace{1mm}Income receipts',
     'LA000035': '\hspace{1mm}Transfer receipts',
     'W163RC': 'Current payments',
     'B021RC': '\hspace{1mm}Imports',
     'A255RC': '\hspace{3mm}Goods',
     'A333RC': '\hspace{5mm}Durable',
     'A340RC': '\hspace{5mm}Nondurable',
     'B656RC': '\hspace{3mm}Services',
     'A655RC': '\hspace{1mm}Income payments',
     'A123RC': '\hspace{1mm}Transfer payments',
     'A124RC': 'Current account balance'}

s = ['A124RC', 'GOODS', 'SERVICES', 'INCOME', 'TRANSFERS']

df = (nipa_df(retrieve_table('T40100')['Data'], n.keys())
      .assign(GOODS = lambda x: x['A253RC'] - x['A255RC'],
              SERVICES = lambda x: x['A646RC'] - x['B656RC'],
              INCOME = lambda x: x['B645RC'] - x['A655RC'],
              TRANSFERS = lambda x: - x['A123RC']))

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0).loc['1989':].multiply(100).round(2))

data.loc['1989':, s].dropna().to_csv(data_dir / 'cab.csv', index_label='date')

node = end_node(data['A124RC'].dropna(), 'black', date='q', 
                full_year=True, percent=True)
write_txt(text_dir / 'cab_node.txt', node)  

cab = abs(data.dropna()['A124RC'].iloc[-1])
tb = abs(data.dropna()['GOODS'].iloc[-1])
ld = dtxt(data.dropna().index[-1])['qtr1']
ltdate = dtxt(data.index[-1])['qtr1']
prdate = dtxt(data.index[-2])['qtr1']
lttbval = abs(data['GOODS'].iloc[-1])
tbprval = abs(data['GOODS'].iloc[-2])
prval = abs(data['A124RC'].iloc[-2])

if pd.isna(data['A124RC'].iloc[-1]) == True:
    mtxt = (f'The initial GDP report for {ltdate} does not include the '+
            'data needed to calculate the current account balance, however, '+
            f'the goods trade deficit for {ltdate} is equivalent to '+
            f'{lttbval:.1f} percent of GDP. ')
else:
    mtxt = (f'In {prdate}, the current account deficit was equivalent to '
            f'{prval:.1f}, and the trade deficit was equivalent to '+
            f'{tbprval:.1f} percent of GDP.')

text = (f'As of {ld}, the US runs a current account deficit of {cab:.1f} '+
        'percent of GDP, primarily as the result of a trade deficit on '+
        f'goods of {tb:.1f} percent of GDP. {mtxt}')
write_txt(text_dir / 'cab.txt', text)
print(text)

As of 2022 Q2, the US runs a current account deficit of 4.0 percent of GDP, primarily as the result of a trade deficit on goods of 5.0 percent of GDP. The initial GDP report for 2022 Q3 does not include the data needed to calculate the current account balance, however, the goods trade deficit for 2022 Q3 is equivalent to 4.4 percent of GDP. 


In [46]:
result = data[n.keys()]
data2 = result.iloc[-6:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data2.columns)]

data2.columns = cols
data2['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data2['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data2.index = data2.index.map(n)
data2 = data2.applymap('{:.2f}'.format).replace('nan', '--')
data2.to_csv(data_dir / 'cab.tex', sep='&', lineterminator='\\\ ', quotechar=' ')

/tmp/ipykernel_22596/3645616702.py:13: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  data2.to_csv(data_dir / 'cab.tex', sep='&', line_terminator='\\\ ', quotechar=' ')


### Federal Interest Outlays share of GDP

In [47]:
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
gdp.index = gdp.index + pd.DateOffset(months=3)
gdp = gdp.A191RC.rolling(4).mean()
tot = fred_df('FYOINT')['VALUE']
tot.index = tot.index + pd.DateOffset(days=1)
df = (tot / gdp).dropna() * 100
fedadj = pd.read_csv(data_dir / 'fedintadj.csv', index_col='date', 
                     parse_dates=True)['RESPPLLOP_N.WW']
res = pd.DataFrame({'Tot': tot, 'FedAdj': fedadj}).fillna(0.0)
adj = res['Tot'] - res['FedAdj']
df2 = (adj / gdp).dropna() * 100
final = pd.DataFrame({'Tot': df, 'Adj': df2})

final.to_csv(data_dir / 'fedintexp.csv', index_label='date')
node = end_node(df, 'magenta', digits=2, percent=True, date='fy', offset=0.15)
node2 = end_node(df2, 'orange', digits=2, percent=True)
write_txt(text_dir / 'fedintexp_nodes.txt', '\n'.join([node, node2]))
val90s = df.loc['1990':'1999'].mean()

text = ('The Office of Management and Budget '+
        '\href{https://www.whitehouse.gov/omb/historical-tables/}{report} '+
        f'\\textbf{{federal interest outlays}} of \${tot.iloc[-1] / 1000:.0f} '+
        f'billion in fiscal year {tot.index[-1].year}, compared to '+
        f'\${tot.iloc[-2] / 1000:.0f} billion in fiscal year '+
        f'{tot.index[-2].year}. Put into the context of the size of '+
        f'the economy, federal interest outlays in fiscal year '+
        f'{df.index[-1].year} were equivalent to {df.iloc[-1]:.2f} '+
        'percent of GDP (see {\color{magenta}\\textbf{---}}), following '+
        f'{df.iloc[-2]:.2f} percent of GDP in FY{df.index[-2].year} '+
        f'and {df.iloc[-3]:.2f} percent in FY{df.index[-3].year}, '+
        f'and compared to an average of {val90s:.1f} percent in '+
        'the 1990s, when interest rates were substantially higher. ')
write_txt(text_dir / 'fedintexp.txt', text)
print(text)

val = (fedadj / 1_000).iloc[-1].round(-1)
adjval = df2.iloc[-1]
cl = c_line('orange')
text = ('The actual interest expense is slightly lower than the reported '+
        'figure, because interest paid to the Federal Reserve gets returned '+
        f'to the Treasury. In FY{df.index[-1].year}, '+
        f'the Fed returned more than \${val:.0f} billion to the Treasury. '+
        f'Adjusting for these remittances, the interest expense was '+
        f'{adjval:.2f} percent of GDP in FY{df.index[-1].year} {cl}. ')
write_txt(text_dir / 'fedintexp2.txt', text)
print('\n', text)

The Office of Management and Budget \href{https://www.whitehouse.gov/omb/historical-tables/}{report} \textbf{federal interest outlays} of \$475 billion in fiscal year 2022, compared to \$352 billion in fiscal year 2021. Put into the context of the size of the economy, federal interest outlays in fiscal year 2022 were equivalent to 1.90 percent of GDP (see {\color{magenta}\textbf{---}}), following 1.55 percent of GDP in FY2021 and 1.64 percent in FY2020, and compared to an average of 2.9 percent in the 1990s, when interest rates were substantially higher. 

 The actual interest expense is slightly lower than the reported figure, because interest paid to the Federal Reserve gets returned to the Treasury. In FY2022, the Fed returned more than \$90 billion to the Treasury. Adjusting for these remittances, the interest expense was 1.54 percent of GDP in FY2022 (see {\color{orange}\textbf{---}}). 


In [48]:
#nipa_series_codes(retrieve_table('T20100'))

### Corporate Profits Destination

In [49]:
s = ['A032RC', 'A438RC', 'A054RC', 'B056RC', 'A127RC']
df = (nipa_df(retrieve_table('T11200')['Data'], s)
         / 1_000_000).dropna().rename({})
df['NNI'] = df['A032RC'] - df['A438RC']
df['TOT'] = df[['A054RC', 'B056RC', 'A127RC']].sum(axis=1)
dt = df.index[-1]
ltdate = dtxt(df.index[-1])['qtr2']
sh = df.div(df.NNI, axis=0) * 100
(sh.loc['1989':, ['A054RC', 'B056RC', 'A127RC']]
   .to_csv(data_dir / 'cprof.csv', index_label='date'))
taxrt = ((df.A054RC / df.TOT) * 100).loc['1989':]
taxrt.to_csv(data_dir / 'cprof_taxrt.csv', index_label='date')
totsh19 = sh.loc['2019', 'TOT'].mean()
divsh19 = sh.loc['2019', 'B056RC'].mean()
resh19 = sh.loc['2019', 'A127RC'].mean()
taxsh19 = sh.loc['2019', 'A054RC'].mean()
taxrt19 = taxrt.loc['2019'].mean()
text = (f'In {ltdate}, corporate profits were '+
        f'\${df.TOT.iloc[-1]:.2f} trillion, equivalent to '+
        f'{sh.TOT.iloc[-1]:.1f} percent of the income paid '+
        'to US nationals after depreciation costs (net national '+
        f'income). Of this, \${df.B056RC.iloc[-1]:.2f} trillion, '+
        f'equivalent to {sh.B056RC.iloc[-1]:.1f} percent of '+
        'net national income, were paid out as dividends '+
        '(see\cbox{blue!70!purple}), '+
        f'\${df.A127RC.iloc[-1] * 1000:,.0f} billion were '+
        'retained (corporate saving, see\cbox{cyan!50!white}), '+
        f'and \${df.A054RC.iloc[-1] * 1000:.0f} billion, '+
        f'{taxrt.iloc[-1]:.1f} percent of corporate profits, '+
        'went to corporate income tax (see\cbox{red!80!orange}). \n\n'+
        f'In 2019, corporate profits were {totsh19:.1f} percent '+
        'of net national income. Dividends were equivalent to '+
        f'{divsh19:.1f} percent, corporate savings were '+
        f'{resh19:.1f} percent, and corporate income taxes were '+
        f'{taxsh19:.1f} percent of net national income '+
        f'and {taxrt19:.1f} percent of corporate profits.')
write_txt(text_dir / 'cprof.txt', text)
print(text)

cols = ['B056RC', 'A127RC', 'A054RC']
sdf = sh[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 1.0)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=3))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}\%}}' 
         for i in cols]
dtv = dtxt(sh.index[-1])['qtr1'].replace(' ', '\\\\ \scriptsize ')
dtn = f'\\absnode{{{{{dtp}}}}}{{{sdf.cumsum().iloc[-1] + 3.0}}}{{\scriptsize{dtv}:}}'
nodes.append(dtn)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'cprof_nodes.txt', nodetext)

In the second quarter of 2022, corporate profits were \$3.00 trillion, equivalent to 15.6 percent of the income paid to US nationals after depreciation costs (net national income). Of this, \$1.74 trillion, equivalent to 9.0 percent of net national income, were paid out as dividends (see\cbox{blue!70!purple}), \$782 billion were retained (corporate saving, see\cbox{cyan!50!white}), and \$479 billion, 15.9 percent of corporate profits, went to corporate income tax (see\cbox{red!80!orange}). 

In 2019, corporate profits were 14.6 percent of net national income. Dividends were equivalent to 9.3 percent, corporate savings were 3.5 percent, and corporate income taxes were 1.8 percent of net national income and 12.4 percent of corporate profits.


### Corporate profits source

In [50]:
s = ['W170RC', 'A262RC', 'W986RC', 'A922RC']
df1 = nipa_df(retrieve_table('T50100')['Data'], s)

s = ['A123RC']
df2 = nipa_df(retrieve_table('T40100')['Data'], s)

s = ['A001RC']
df3 = nipa_df(retrieve_table('T10705')['Data'], s)

cprof = pd.DataFrame()
cprof['ROW Saving'] = (df2['A123RC'] / df3['A001RC']) * 100
cprof['HH Saving'] = (- df1['W986RC'] / df3['A001RC']) * 100
cprof['Gov Saving'] = (- df1['A922RC'] / df3['A001RC']) * 100
cprof['Investment'] = ((df1['W170RC'] - df1['A262RC']) / df3['A001RC']) * 100

cprof.loc['1989':].to_csv(data_dir / 'cprof2.csv', index_label='date')

In [51]:
#nipa_series_codes(retrieve_table('T50100'))

### Labor Productivity ST / LT

In [2]:
df = fred_df('OPHNFB', start='1968')['VALUE']
df = growth_rate(df)
lt = df.rolling(80).mean().dropna()
st = df.rolling(4).mean().dropna()
data = pd.DataFrame({'ST': st, 'LT': lt})
data.loc['1989':].to_csv(data_dir / 'prod_st_lt.csv', index_label='date')
ltdt = dtxt(data.index[-1])['qtr1']
ltavg = data['LT'].mean()
lt = data.iloc[-1]
colors = {'LT': 'orange!80!yellow', 'ST': 'blue!80!black'}
cl = {name: c_line(col) for name, col in colors.items()}
text = ('Over the longer-term, US labor productivity growth '+
        f'has averaged {ltavg:.1f} percent per year. The trailing 20-'+
        f'year average growth rate was {lt["LT"]:.1f} percent in {ltdt} '+
        f'{cl["LT"]}. During the 1990s and early 2000s, labor '+
        'productivity growth was above its long-term average. '+
        'In contrast, from 2010 to 2017, productivity growth '+
        f'was below average. Over the year ending {ltdt}, productivity '+
        f'growth averaged {lt["ST"]:.1f} percent {cl["ST"]}. ')
write_txt(text_dir / 'prod_st_lt.txt', text)
print(text)

Over the longer-term, US labor productivity growth has averaged 2.0 percent per year. The trailing 20-year average growth rate was 1.7 percent in 2022 Q3 (see {\color{orange!80!yellow}\textbf{---}}). During the 1990s and early 2000s, labor productivity growth was above its long-term average. In contrast, from 2010 to 2017, productivity growth was below average. Over the year ending 2022 Q3, productivity growth averaged -1.3 percent (see {\color{blue!80!black}\textbf{---}}). 


### Labor Productivity

In [3]:
# Series stored as a dictionary
series = {'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output',
          'PRS85006033': 'hours_index'}

In [4]:
# Start year and end year
dates = (1989, 2023)
df = bls_api(series, dates, bls_key)

df.to_csv(data_dir / 'lprod.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [5]:
df = pd.read_csv(data_dir / 'lprod.csv', 
                 index_col='date', parse_dates=True)
ltdt = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
d = series_info(df['value'])
s = {srs: {'lt': value_text(df[srs].iloc[-1], adj='annual'), 
           'pr': value_text(df[srs].iloc[-2], adj='annual'), 
           'lt2': value_text(df[srs].iloc[-1], style='increase_of'), 
           'pr2': value_text(df[srs].iloc[-2]),
           'pr3': value_text(df[srs].iloc[-2], style='increase_of')} 
     for srs in series.values()}
compare = compare_text(d['five_year_mean'], d['mean'], [0.1, 0.5, 2.0])

text = (f'In {ltdt}, labor productivity {s["value"]["lt"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["lt2"]} in '+
        f'real output and {s["hours"]["lt2"]} in hours worked. In the '+
        f'prior quarter, {prdt}, labor productivity {s["value"]["pr"]}, '+
        f'as real output {s["output"]["pr2"]} and hours of work '+
        f'{s["hours"]["pr2"]}. Over the past five years, labor productivity '+
        f'growth has averaged {d["five_year_mean"]:.1f} percent, '+
        f'{compare} the 1989-onward average of {d["mean"]:.1f} percent.')
write_txt(text_dir / 'lprod.txt', text)
print(text, '\n')

hr19 = value_text(df.loc['2017': '2019', 'hours'].mean(), adj='average')
ch19 = (df['hours_index'].iloc[-1] / 
        df.loc['2019-10-01', 'hours_index'] - 1) * 100
ch19txt = value_text(ch19, 'increase_by', adj='total')
text = (f'Total hours worked in nonfarm businesses {s["hours"]["lt"]} '+
        f'in {ltdt}, following {s["hours"]["pr3"]} in {prdt}. From '+
        f'2017 through 2019, total hours worked {hr19}. Since 2019, '+
        f'hours worked have {ch19txt}.')
write_txt(text_dir / 'tot_hours.txt', text)
print(text)

In 2022 Q3, labor productivity increased at an annual rate of 0.3 percent (see\cbox{teal}), as the result of an increase of 2.8 percent in real output and an increase of 2.4 percent in hours worked. In the prior quarter, 2022 Q2, labor productivity decreased at an annual rate of 4.1 percent, as real output decreased 1.2 percent and hours of work increased 2.9 percent. Over the past five years, labor productivity growth has averaged 1.7 percent, slightly below the 1989-onward average of 1.9 percent. 

Total hours worked in nonfarm businesses increased at an annual rate of 2.4 percent in 2022 Q3, following an increase of 2.9 percent in 2022 Q2. From 2017 through 2019, total hours worked increased at an average rate of 1.4 percent. Since 2019, hours worked have increased by a total of 1.3 percent.


### Gross Labor Income

In [6]:
s = {'LNS12005054': 'Hours', 'LNS12000000': 'Employment'}
df = bls_api(s, (1989, 2023), bls_key)
df['Total'] = df['Hours'] * df['Employment']
df.to_csv(data_dir / 'emp_hrs.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [15]:
emp = (pd.read_csv(data_dir / 'emp_hrs.csv', parse_dates=[0])
         .set_index('date')['Total'])
coe = nipa_df(retrieve_table('T20100')['Data'], ['A033RC'])
pce = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])
data = coe.join(pce).join(emp.resample('QS').mean()).dropna()
data['real_coe'] = data['A033RC'] / (data['DPCERG'] / data['DPCERG'].iloc[-1])
data['coe_inp'] = data['real_coe'] / data['Total']
data['wage'] = data['coe_inp'] * data['Total'].iloc[0]
data['work'] = data['real_coe'] - data['wage']

# Calculate contributions to growth
result = growth_contrib(data, 'real_coe')[['work', 'wage']]
result.dropna().loc['1989':]
result.to_csv(data_dir / 'gli.csv', index_label='date')

ltdate = dtxt(result.index[-1])['qtr1']
totval = result.iloc[-1].sum()
wage = result['wage'].iloc[-1]
work = result['work'].iloc[-1]

incdec = 'increased' if totval > 0 else 'decreased'
txt1 = (f'{incdec} at an annualized and inflation-adjusted '+
        f'rate of {abs(totval):.2f} percent')

if totval == 0:
    txt1 = 'was unchanged, after adjusting for inflation,'
    
txt2 = value_text(wage, 'contribution', 'pp', digits=2)
txt3 = value_text(work, 'contribution', 'pp', digits=2)

text = (f'{txt1} in {ltdate}. Changes in wages {txt2}, '+
        f'and changes in total hours worked {txt3}.')
write_txt(text_dir / 'gli.txt', text)
print(text)

increased at an annualized and inflation-adjusted rate of 2.64 percent in 2022 Q3. Changes in wages contributed 1.66 percentage points, and changes in total hours worked contributed 0.98 percentage point.


### Financial Obligations Ratio

In [6]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=FOR&series=1dc13603606b1a2cf3c07004eeb7f026&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d = {k: v.replace(', seasonally adjusted', '') for k, v in d.items()}

data = clean_data.rename(d, axis=1)
data.to_csv(data_dir / 'for.csv', index_label='date')
col1 = 'blue!80!black'
node = end_node(data['Financial obligations ratio'], col1, 
                percent=True, date='q', full_year=True)
write_txt(text_dir / 'for_node.txt', node)

ltdate = dtxt(data.index[-1])['qtr1']
ltval = data['Financial obligations ratio'].iloc[-1]
col2 = 'red'
node2 = end_node(data['Debt service ratio'], col2, percent=True)
write_txt(text_dir / 'dsr_node.txt', node2)

dsrval = data['Debt service ratio'].iloc[-1]

text = (f'As of {ltdate}, the \\textbf{{financial obligations ratio}} '+
        f'is {ltval:.1f} percent {c_line(col1)}, and the '+
        f'\\textbf{{debt service ratio}} is {dsrval:.1f} percent '+
        f'{c_line(col2)}.')
write_txt(text_dir / 'for.txt', text)
print(text)

As of 2022 Q2, the \textbf{financial obligations ratio} is 14.3 percent (see {\color{blue!80!black}\textbf{---}}), and the \textbf{debt service ratio} is 9.6 percent (see {\color{red}\textbf{---}}).


### Shiller real return trailing 20-year average

In [7]:
url = 'http://www.econ.yale.edu/~shiller/data/ie_data.xls'
data = pd.read_excel(url, sheet_name='Data', header=7, 
                     index_col='Date').dropna(subset=['TR CAPE'])
data.index = pd.to_datetime(data.index.format())
data.loc['1989':, 'TR CAPE'].to_csv(data_dir / 'catrpe.csv', 
                                    index_label='date')
color = 'blue!80!cyan'
cl = c_line(color)
pe = data.loc['1989':, 'TR CAPE']
ltdt = dtxt(pe.index[-1])['mon1']
prdt = dtxt(pe.index[-2])['mon1']
yrdt = dtxt(pe.index[-13])['mon1']
ltval = pe.iloc[-1]
prval = pe.iloc[-2]
yrval = pe.iloc[-13]
val19 = pe.loc['2019'].mean()
val00 = pe.loc['2000'].mean()

text = (f'In {ltdt}, the Shiller total return CAPE ratio was '+
        f'{ltval:.1f}, compared to {prval:.1f} in {prdt} and '+
        f'{yrval:.1f} in {yrdt} {cl}. In 2019, the Shiller CAPE ratio '+
        f'was {val19:.1f}, on average. In 2000, during the stock market '+
        f'bubble, the ratio averaged {val00:.1f}. ')
write_txt(text_dir / 'cape.txt', text)
print(text)
node = end_node(data['TR CAPE'], color, date='m')
write_txt(text_dir / 'cape_node.txt', node)
col = ['Price', 'Dividend']
df = data.loc['1960':, col].dropna()
for yrs in [10, 15, 20]:
    mos = yrs * 12
    dy = (df.Dividend / df.Price).rolling(mos).mean()
    pch = (df.Price.pct_change(mos)+1)**(1/yrs) - 1
    df[f'r{yrs}'] = (dy + pch) * 100
    
res = df.loc['1989':, ~df.columns.isin(col)]
res.to_csv(data_dir / f'sp500rr.csv', index_label='date', 
           float_format='%g')
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'r20': 'green!80!blue', 
          'r15': 'orange',
          'r10': 'blue'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], 
                            percent=True, full_year=True, 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'sp500rr_nodes.txt', nodes)  
url = 'http://www.econ.yale.edu/~shiller/data.htm'
lt20 = df['r20'].iloc[-1]
ltdt = dtxt(df.index[-1])['mon1']
pr = df['r20'].loc['1995':'2005'].mean()
lt10 = df['r10'].iloc[-1]
pr10 = df['r10'].loc['1995':'2005'].mean()
cl = c_line(colors['r20'])
text = (f'According to historical stock market \href{{{url}}}'+
        '{data} from Robert Shiller, the \\textbf{inflation-adjusted '+
        'trailing twenty-year annual rate of return} of the S\&P '+
        f'500 was {lt20:.1f} percent as of {ltdt} {cl}. Ultra-long-'+
        'term real returns are currently low relative to the '+
        'average trailing twenty-year real annual return of '+
        f'{pr:.1f} percent during 1995--2005. The trailing ten-'+
        f'year real return was {lt10:.1f} percent, as of {ltdt}, '+
        f'and {pr10:.1f} percent during 1995--2005 '+
        f'{c_line(colors["r10"])}. ')
write_txt(text_dir / 'sp500rr.txt', text)
print(text)

In November 2022, the Shiller total return CAPE ratio was 29.6, compared to 29.4 in October 2022 and 42.1 in November 2021 (see {\color{blue!80!cyan}\textbf{---}}). In 2019, the Shiller CAPE ratio was 32.1, on average. In 2000, during the stock market bubble, the ratio averaged 45.1. 
According to historical stock market \href{http://www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the \textbf{inflation-adjusted trailing twenty-year annual rate of return} of the S\&P 500 was 7.0 percent as of September 2022 (see {\color{green!80!blue}\textbf{---}}). Ultra-long-term real returns are currently low relative to the average trailing twenty-year real annual return of 10.1 percent during 1995--2005. The trailing ten-year real return was 9.5 percent, as of September 2022, and 10.7 percent during 1995--2005 (see {\color{blue}\textbf{---}}). 


In [8]:
dy = ((data.D / data.P).dropna() * 100).loc['1989':].rename('DY')
dy.to_csv(data_dir / 'sp500dy.csv', index_label='date')
color = 'green!80!black'

node = end_node(dy, color, date='m', full_year=True, percent=True, 
                digits=2, offset=0.3)
write_txt(text_dir / 'sp500dy_node.txt', node)

ltdt = dtxt(dy.index[-1])['mon1']
prdt = dtxt(dy.index[-2])['mon1']
yrdt = dtxt(dy.index[-13])['mon1']
ltval = dy.iloc[-1]
prval = dy.iloc[-2]
yrval = dy.iloc[-13]
cl = c_line(color)
avg = dy.loc['1990': '2015'].mean()
text = (f'In {ltdt}, the dividend yield for the S\&P 500 was '+
        f'{ltval:.2f} percent {cl}, compared to {prval:.2f} percent '+
        f'in {prdt}, and {yrval:.2f} percent in {yrdt}. From 1990 to '+
        f'2015, the dividend yield averaged {avg:.2f} percent.')
write_txt(text_dir / 'sp500div.txt', text)
print(text)

In September 2022, the dividend yield for the S\&P 500 was 1.70 percent (see {\color{green!80!black}\textbf{---}}), compared to 1.56 percent in August 2022, and 1.33 percent in September 2021. From 1990 to 2015, the dividend yield averaged 2.09 percent.


### S&P 500 Quarterly Data

In [61]:
#url = ('https://us.spindices.com/documents/'+
#       'additional-material/sp-500-eps-est.xlsx')


### High Quality Corporate Bond Yield

In [9]:
df = fred_df('HQMCB10YR')
df.to_csv(data_dir / 'hqcb.csv', index_label='date')

color = 'violet!80!black'
node = end_node(df['VALUE'], color, date='m', percent=True, 
                full_year=True, digits=2)
write_txt(text_dir / 'hqcb_node.txt', node)

data = df.VALUE.apply('{:.2f} percent'.format).values
idx = [dtxt(i)['mon1'] for i in df.index]
cl = c_line(color)

text = (f'The yield on high-quality corporate bonds with a maturity '+
        f'of 10 years is {data[-1]} in {idx[-1]}, following '+
        f'{data[-2]} in {idx[-2]} {cl}. One year prior, in '+
        f'{idx[-13]}, this spot rate was {data[-13]}, and '+
        f'three years prior, in {idx[-37]}, it was {data[-37]}.')
write_txt(text_dir / 'hqcb.txt', text)
print(text)

The yield on high-quality corporate bonds with a maturity of 10 years is 5.83 percent in October 2022, following 5.27 percent in September 2022 (see {\color{violet!80!black}\textbf{---}}). One year prior, in October 2021, this spot rate was 2.63 percent, and three years prior, in October 2019, it was 2.91 percent.


### Treasury Monthly Data

In [11]:
cols = ['record_date', 'classification_desc', 
        'current_month_rcpt_outly_amt']
fields = ','.join(cols)
lines = ','.join(['2', '3', '5', '6', '7', '12'])
url = ('https://api.fiscaldata.treasury.gov/services/api/'+
       'fiscal_service/v1/accounting/mts/mts_table_9'+
       f'?filter=src_line_nbr:in:({lines})&format=csv&'+
       f'fields={fields}&page[size]=600')

other = lambda x: x['Total'] - x['Individual Income Taxes']
si_cols = ['Employment and General Retirement', 'Other Retirement', 
           'Unemployment Insurance']
si = lambda x: x[si_cols].sum(axis=1)
df = (pd.read_csv(url, index_col=cols[:2], parse_dates=True)
        ['current_month_rcpt_outly_amt']
        .unstack().astype('float').divide(1_000_000_000_000)
        .assign(Other = other, SI = si).rolling(12).sum().dropna())
df.to_csv(data_dir / 'tmb_rec.csv', float_format='%g', 
          index_label='date')

In [12]:
df = pd.read_csv(data_dir / 'tmb_rec.csv', index_col='date', 
                 parse_dates=True)
col_iit = 'blue!70!black'
write_txt(text_dir / 'tmb_rec_iit.txt', 
          end_node(df['Individual Income Taxes'], col_iit))
cl_iit = c_line(col_iit)
col_oth = 'blue!40!cyan'
write_txt(text_dir / 'tmb_rec_oth.txt', 
          end_node(df['Other'], col_oth))
cl_oth = c_line(col_oth)

# Text 
url = 'https://fiscal.treasury.gov/reports-statements/mts/current.html'
tval = df.iloc[-1].apply(lambda x: f'\${x:.1f} trillion')
ltdt = dtxt(df.index[-1])['mon1']
text = (f'The United States Treasury \href{{{url}}}{{report}} '+
        'federal government receipts and outlays in the Monthly '+
        f'Treasury Statement. 'f'Over the 12 months ending {ltdt}, '+
        f'\\textbf{{Federal government receipts}} total {tval["Total"]}, '+
        f'of which {tval["Individual Income Taxes"]} are from '+
        f'individual income taxes {cl_iit}. The remaining receipts '+
        f'{cl_oth} are largely social insurance contributions '+
        f'({tval["SI"]}) and corporate income taxes '+
        f'({tval["Corporation Income Taxes"]}).')
write_txt(text_dir / 'tmb_rec.txt', text)
print(text)

The United States Treasury \href{https://fiscal.treasury.gov/reports-statements/mts/current.html}{report} federal government receipts and outlays in the Monthly Treasury Statement. Over the 12 months ending October 2022, \textbf{Federal government receipts} total \$4.9 trillion, of which \$2.7 trillion are from individual income taxes (see {\color{blue!70!black}\textbf{---}}). The remaining receipts (see {\color{blue!40!cyan}\textbf{---}}) are largely social insurance contributions (\$1.5 trillion) and corporate income taxes (\$0.4 trillion).


### International Investment Position (IIP)

In [14]:
# Annual GDP for 1988-2005
s = 'A191RC'
r = bea_api_nipa([f'T10105'], bea_key, freq='A')
data = json.loads(r[0][2])['BEAAPI']['Results']
date = lambda x: (pd.to_datetime(x.TimePeriod) + 
                  pd.DateOffset(months=6))
value = lambda x: x.DataValue.str.replace(',','')
gdpa = (pd.DataFrame(data['Data']).query('SeriesCode == @s')
          .assign(date = date, value=value).set_index('date')
          .loc[:'2005', 'value'].astype(int))
gdpq = nipa_df(retrieve_table('T10105')['Data'], [s])[s]
gdp = pd.concat([gdpa, gdpq.loc['2006':]])

srs = ['FinAssets', 'FinLiabs', 'Net']
years = ','.join(map(str, range(1988, 2023)))
res = pd.DataFrame()
for s in srs:
    url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
           f'&method=GetData&datasetname=IIP&TypeOfInvestment={s}'+
           f'&Component=Pos&Frequency=A,QNSA&Year={years}')
    r = requests.get(url)
    t = pd.DataFrame(r.json()['BEAAPI']['Data'])
    a = t.query('Frequency == "A"')[['TimePeriod', 'DataValue']]
    a = a.set_index(pd.to_datetime(a['TimePeriod']) + 
                    pd.DateOffset(months=6)).loc[:'2005', 'DataValue']
    q = t.query('Frequency == "QNSA"')[['TimePeriod', 'DataValue']]
    q = q.set_index(pd.to_datetime(q['TimePeriod']))['DataValue']
    res[s] = pd.concat([a, q]).astype('float')

sh = res.divide(gdp, axis=0).multiply(100).dropna()
sh.to_csv(data_dir / 'iip.csv', index_label='date')

tot = res.iloc[-1] / 1_000_000
lt = sh.iloc[-1]
ltdt = dtxt(sh.index[-1])['qtr1']
pr = sh.iloc[-2]
prdt = dtxt(sh.index[-2])['qtr1']
pr19 = sh.loc['2019'].mean()
col_a, col_l, col_n = 'blue!95!violet', 'red', 'cyan!25!white'

text = (f'In {ltdt}, domestic holdings of foreign assets total '+
        f'\${tot.FinAssets:.1f} trillion, equivalent to {lt.FinAssets:.1f} '+
        f'percent of GDP {c_line(col_a)}. In {prdt}, these assets were '+
        f'equivalent to {pr.FinAssets:.1f} percent of GDP, and in '+
        f'2019, they were equivalent to {pr19.FinAssets:.1f} percent. '+
        'Domestic liabilities to the foreign sector total '+
        f'\${tot.FinLiabs:.1f} trillion, or {lt.FinLiabs:.1f} percent of '+
        f'GDP, in {ltdt}, following {pr.FinLiabs:.1f} percent in {prdt}, '+
        f'and {pr19.FinLiabs:.1f} percent in 2019 {c_line(col_l)}.\n\n'+
        f'The overall result of these financial positions, net IIP, or '+
        'holdings of foreign assets minus liabilities, identifies the '+
        'US as a net debtor to the rest of the world, to the equivalent '+
        f'of {abs(lt.Net):.1f} percent of GDP in {ltdt}, following '
        f'{abs(pr.Net):.1f} percent in {prdt}, and {abs(pr19.Net):.1f} '+
        f'percent in 2019 {c_box(col_n)}.')
write_txt(text_dir / 'niip.txt', text)
print(text)

In 2022 Q2, domestic holdings of foreign assets total \$31.0 trillion, equivalent to 122.7 percent of GDP (see {\color{blue!95!violet}\textbf{---}}). In 2022 Q1, these assets were equivalent to 137.6 percent of GDP, and in 2019, they were equivalent to 129.7 percent. Domestic liabilities to the foreign sector total \$47.3 trillion, or 187.3 percent of GDP, in 2022 Q2, following 209.3 percent in 2022 Q1, and 181.4 percent in 2019 (see {\color{red}\textbf{---}}).

The overall result of these financial positions, net IIP, or holdings of foreign assets minus liabilities, identifies the US as a net debtor to the rest of the world, to the equivalent of 64.6 percent of GDP in 2022 Q2, following 71.7 percent in 2022 Q1, and 51.7 percent in 2019 (see \cbox{cyan!25!white}).


### H.6 Money Stock - M2 (Monthly)

In [13]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H6&series=411c4c269dc600450339f8d4809d80eb&lastobs=&'+
       'from=01/01/1987&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
df.rename(d, axis=1).to_csv(data_dir / 'h6raw.csv', index_label='date')

In [14]:
df = pd.read_csv(data_dir / 'h6raw.csv', index_col='date', 
                 parse_dates=True)
ltdate = dtxt(df.index[-1])['mon1']
prmodt = dtxt(df.index[-2])['mon1']
ltval = df['M2'].iloc[-1] / 1000.0
one_yr = value_text(df['M2'].pct_change(12).iloc[-1] * 100)
pr_mo = value_text(df['M2'].pct_change(12).iloc[-2] * 100, 
                   style='increase_of')
three_yr = value_text(df['M2'].pct_change(36).iloc[-1] * 100)

# M2 share of GDP
m2q = df.M2.resample('QS').mean() * 1_000
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2gdp = ((m2q / gdp) * 100).dropna()
m2gi = (m2gdp / m2gdp.iloc[0]).rename('value')
m2gi.to_csv(data_dir / 'm2gdp.csv', index_label='date',
         float_format='%g')
ltqtr = dtxt(m2gi.index[-1])['qtr2']
lt = m2gdp.iloc[-1]
totch = (m2gi.iloc[-1] - 1) * 100

txt1 = (f'In {ltdate}, the M2 money stock totals \${ltval:.1f} '+
        'trillion. Put into the context of overall economic activity, '+
        f'M2 is equivalent to {lt:.1f} percent of GDP in {ltqtr}. During '+
        'the 1990s, the ratio of money to economic activity was falling '+
        f'{c_line("magenta")}. Following the great recession, the money '+
        'supply has expanded relative to activity. Since 1989, the ratio '+
        f'has increased by a total of {totch:.1f} percent.')
write_txt(text_dir / 'm2lvl.txt', txt1)
col = 'green!80!blue'

mo_ch = value_text((df.M2.pct_change()*100).iloc[-1], threshold=0.1)
pc_mo_ch = value_text((df.M2.pct_change()*100).iloc[-2], 
                      'increase_of', threshold=0.1)
prdt = dtxt(df.index[-2])['mon1']
txt2 = (f'The M2 money stock {mo_ch} in {ltdate}, over the previous '+
        f'month, following {pc_mo_ch} in {prdt}. Over the past 12 '+
        f'months, the money stock {one_yr} {c_line(col)}. The M2 money stock has {three_yr}, '+
        'in total, over the past three years. ')
write_txt(text_dir / 'm2chg.txt', txt2)
print(txt1, '\n', txt2)

r = pd.DataFrame({'value': df['M2'].pct_change(12) * 100,
                  '3M': m3rate(df['M2']),
                  '1M': (((df.M2.pct_change() + 1) ** 12) - 1) * 100}
                ).loc['1989':]
r.value.to_csv(data_dir / 'm2.csv', index_label='date', header=True,
         float_format='%g')

node = end_node(r.value, col, percent=True, 
                date='m', full_year=True)
write_txt(text_dir / 'm2_node.txt', node)

In September 2022, the M2 money stock totals \$21.5 trillion. Put into the context of overall economic activity, M2 is equivalent to 83.9 percent of GDP in the third quarter of 2022. During the 1990s, the ratio of money to economic activity was falling (see {\color{magenta}\textbf{---}}). Following the great recession, the money supply has expanded relative to activity. Since 1989, the ratio has increased by a total of 48.4 percent. 
 The M2 money stock decreased 0.4 percent in September 2022, over the previous month, following a decrease of 0.2 percent in August 2022. Over the past 12 months, the money stock increased 2.5 percent (see {\color{green!80!blue}\textbf{---}}). The M2 money stock has increased 43.2 percent, in total, over the past three years. 


### Consumer Credit (G.19)

The same chart also includes a line from the Z.1 notebook

In [2]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G19&series=8c7511a37e4aea678be81e7a61df57db&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, df = clean_fed_data(url)
cc = df['DTCTL.M']

dpi = (pd.read_csv(data_dir / 'nipa20600.csv', index_col='date', 
                   parse_dates=True).loc['1989':, 'A067RC'])
rate = ((cc / dpi) * 100).rename('value').dropna()
rate.to_csv(data_dir / 'cc_dpi_monthly.csv', index_label='date')

node_color = 'green!60!blue!80!black'
node = end_node(rate, node_color, date='m', percent=True, full_year=True)
write_txt(text_dir / 'cc_dpi_monthly_node.txt', node)

ltdate = dtxt(cc.index[-1])['mon1']
ltval = cc.iloc[-1] / 1_000_000
ltrate = rate.iloc[-1]
one_yr = value_text(cc.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_dpi = value_text(dpi.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_rate = value_text(rate.diff(12).iloc[-1], adj='total', 
                         ptype='pp', threshold=0.1)
cline = c_line(node_color)
also = 'also ' if one_yr == one_yr_dpi else ''    
text = ('In the monthly measure, consumer credit totals '+
        f'\${ltval:.2f} trillion US dollars on a '+
        f'seasonally-adjusted and annualized basis in {ltdate}. '+
        f'Over the past year, consumer credit {one_yr}, while '+
        f'after-tax income {also}{one_yr_dpi}. As a result, the ratio '+
        f'of consumer credit to disposable income {one_yr_rate}. '+
        f'In {ltdate}, total consumer credit is equivalent to '+
        f'{ltrate:.1f} percent of annualized {ltdate} disposable '+
        f'income {cline}. ')
write_txt(text_dir / 'cc_dpi.txt', text)
print(text)

In the monthly measure, consumer credit totals \$4.70 trillion US dollars on a seasonally-adjusted and annualized basis in September 2022. Over the past year, consumer credit increased by 7.9 percent, while after-tax income increased by 3.2 percent. As a result, the ratio of consumer credit to disposable income increased by a total of 1.1 percentage points. In September 2022, total consumer credit is equivalent to 25.0 percent of annualized September 2022 disposable income (see {\color{green!60!blue!80!black}\textbf{---}}). 


### Employment Cost Index

In [18]:
series = {'CIU2020000000000A': 'WS',
          'CIU2030000000000A': 'Be'}

dates = (2001, 2023)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir/ 'eci.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [19]:
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)
adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'WS': 'green!80!blue', 
          'Be': 'cyan!40!blue'}
date = {series: 'q' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            percent=True, colon=False,
                            size=1.1, offset=adj[series])
                   for series, color in colors.items()])
write_txt(text_dir / 'eci_nodes.txt', nodes)

obs = [(-1, 'lt'), (-2, 'pr'), (-3, 'pr2')]
sty = [('increase', '1'), ('increase_of', '2'), 
       ('increase_by', '3')]
dt = {n: dtxt(df.index[i]) for i, n in obs}
v = {f'{name}_{n}_{s}': value_text(val, style=style) 
     for i, n in obs for name, val 
     in df.iloc[i].to_dict().items() 
     for style, s in sty}

val19 = {s: value_text(df.loc['2019', s].mean(), 'increase_by') 
         for s in ['WS', 'Be']}
text = (f'In {dt["lt"]["qtr2"]}, private industry wage and '+
        f'salary costs {v["WS_lt_3"]} (one-year percent '+
        f'change, {c_line(colors["WS"]).replace("(", "")}, '+
        f'following {v["WS_pr_2"]} in {dt["pr"]["qtr1"]}, '+
        f'and {v["WS_pr2_2"]} in {dt["pr2"]["qtr1"]}. In '+
        f'2019, private wages and salaries costs {val19["WS"]}, '+
        'on average.\n\nThe cost of private sector benefits '+
        f'{v["Be_lt_3"]} {c_line(colors["Be"])} over the year '+
        f'ending {dt["lt"]["qtr1"]}, following {v["Be_pr_2"]} '+
        f'in {dt["pr"]["qtr1"]}. In 2019, private-sector '+
        f'benefits costs {val19["Be"]}, on average.')
write_txt(text_dir / 'eci.txt', text)
print(text)

In the third quarter of 2022, private industry wage and salary costs increased by 5.2 percent (one-year percent change, see {\color{green!80!blue}\textbf{---}}), following an increase of 5.7 percent in 2022 Q2, and an increase of five percent in 2022 Q1. In 2019, private wages and salaries costs increased by three percent, on average.

The cost of private sector benefits increased by five percent (see {\color{cyan!40!blue}\textbf{---}}) over the year ending 2022 Q3, following an increase of 5.3 percent in 2022 Q2. In 2019, private-sector benefits costs increased by two percent, on average.


### Treasury International Capital

In [2]:
url = ('https://www.treasury.gov/resource-center/data-chart-center/'+
       'tic/Documents/s1_99996.txt')
df = pd.read_table(url, header=9, delimiter=' ', skipinitialspace=True, 
                   index_col=0).iloc[1:]
df = df.set_index(pd.to_datetime(df['Year-Mo'])).drop('Year-Mo', axis=1)
df.columns = ['P_Treasury_Bonds', 'P_Agency_Bonds', 'P_Corporate_Bonds', 
              'P_Corporate_Stocks', 'P_Foreign_Bonds', 'P_Foreign_Stocks',
              'S_Treasury_Bonds', 'S_Agency_Bonds', 'S_Corporate_Bonds', 
              'S_Corporate_Stocks', 'S_Foreign_Bonds', 'S_Foreign_Stocks']
df = df.replace(',','', regex=True).astype(float).sort_index()
df.loc['1988':].to_csv(data_dir/ 'tic.csv', index_label='date')

In [3]:
df = pd.read_csv(data_dir / 'tic.csv', index_col='date', 
                 parse_dates=True)
s = ['Treasury_Bonds', 'Agency_Bonds', 'Corporate_Bonds']
for i in s:
    df[i] = df[f'P_{i}'] - df[f'S_{i}']

pce = pd.read_csv(data_dir / 'nipa20804.csv', index_col='date', 
                  parse_dates=True).loc[df.index, 'DPCERG']
pr = pce / pce.iloc[-1]
data = (df[s].divide(pr, axis=0).rolling(12).sum()
             .loc['1989':].divide(1000).dropna())
data.to_csv(data_dir/ 'tic_bond.csv', index_label='date')
date = dtxt(data.index[-1])['mon1']
write_txt(text_dir / 'tic_date.txt', date)
print(date)

September 2022


In [4]:
ltdt = dtxt(data.index[-1])['mon1']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
tbgdp = ((data.iloc[-1].loc['Treasury_Bonds'] * 1_000) / 
         gdp.iloc[-1]) * 100

colors = {'Treasury_Bonds': 'black!25!white', 
          'Corporate_Bonds': 'blue!70!white',
          'Agency_Bonds': 'orange!30!yellow!90!white'}

cb = {k: c_box(v) for k,v in colors.items()}

d = {}
for i in s:
    item = i.replace('_', ' ').lower()
    val = data.iloc[-1].loc[i]
    buysell = 'buyer' if val > 0 else 'seller'
    txt = (f'a net {buysell} of \${val:,.0f} billion of US {item}' 
           if abs(val) >= 5 else 
           f'about unchanged in holdings of US {item}')
    d[i] = txt

text = (f'Over the year ending {ltdt}, the rest of the world was '+
        f'{d["Treasury_Bonds"]}, equivalent to {tbgdp:.1f} percent '+
        f'of US GDP {cb["Treasury_Bonds"]}. Over the same period, '+
        f'the rest of the world was {d["Agency_Bonds"]}, '+
        f'{cb["Agency_Bonds"]}, and {d["Corporate_Bonds"]}, '+
        f'{cb["Corporate_Bonds"]}.')
write_txt(text_dir / 'tic.txt', text)
print(text)

Over the year ending September 2022, the rest of the world was a net buyer of \$696 billion of US treasury bonds, equivalent to 2.7 percent of US GDP (see \cbox{black!25!white}). Over the same period, the rest of the world was a net buyer of \$307 billion of US agency bonds, (see \cbox{orange!30!yellow!90!white}), and a net buyer of \$147 billion of US corporate bonds, (see \cbox{blue!70!white}).


### FRBNY Survey of Consumer Expectations Job Separation Expectation

In [10]:
url = ('https://www.newyorkfed.org/medialibrary/interactives/'+
       'sce/sce/downloads/data/frbny-sce-data.xlsx')
df = pd.read_excel(url, sheet_name='Job separation expectation', 
                   skiprows=3, index_col=0)
df.index = pd.to_datetime([f'{str(i)[:4]}-{str(i)[-2:]}-01' 
                          for i in df.index])
df.columns = ['Losing', 'Leaving']
df.to_csv(data_dir / 'sce_job_separation.csv', index_label='date')

In [11]:
df = pd.read_csv(data_dir / 'sce_job_separation.csv', index_col='date', 
                 parse_dates=True)

lcol = 'cyan'
vcol = 'violet!50!blue'
vnode = end_node(df.Leaving, vcol, date='m', percent=True, 
                 full_year=True, colon=False, offset=0.35)
lnode = end_node(df.Losing, lcol, percent=True)
nodes = '\n'.join([vnode, lnode])
write_txt(text_dir / 'sce_job_sep_nodes.txt', nodes)

ltdt = dtxt(df.index[-1])['mon1']
ltv = df.Leaving.iloc[-1]
prv = df.Leaving.loc['2019'].mean()
ltl = df.Losing.iloc[-1]
prl = df.Losing.loc['2019'].mean()
diff = ltv - ltl
diffpr = prv - prl
exfb = 'exceed' if diff > 0 else 'false below'
text = (f'In {ltdt}, the perceived likelihood of leaving '+
        "one's job voluntarily in the next 12 months "+
        f'averages {ltv:.1f} percent, compared to {prv:.1f} '+
        f'percent in 2019 {c_line(vcol)}. In the latest month, '+
        f"the perceived probability losing one's job is {ltl:.1f} "+
        f'percent, compared to {prl:.1f} percent in 2019 '+
        f'{c_line(lcol)}.\n\nDuring the pandemic, in April 2020, '+
        'job loss expectations exceeded job leaving expectations. '+
        f'In {ltdt}, job leaving expectations {exfb} job loss '+
        f'expectations by {diff:.1f} percentage points, compared '+
        f'to {diffpr:.1f} percentage points in 2019. ')
write_txt(text_dir / 'sce_job_sep.txt', text)
print(text)

In October 2022, the perceived likelihood of leaving one's job voluntarily in the next 12 months averages 19.5 percent, compared to 21.0 percent in 2019 (see {\color{violet!50!blue}\textbf{---}}). In the latest month, the perceived probability losing one's job is 12.0 percent, compared to 14.3 percent in 2019 (see {\color{cyan}\textbf{---}}).

During the pandemic, in April 2020, job loss expectations exceeded job leaving expectations. In October 2022, job leaving expectations exceed job loss expectations by 7.5 percentage points, compared to 6.8 percentage points in 2019. 


### Real Effective Exchange Rate

From BIS

Index 2010 = 100

In [5]:
df = fred_df('RBUSBIS')
df.to_csv(data_dir / 'reer.csv', index_label='date')
ltdt = dtxt(df.index[-1])['mon1']
ltval = value_text(df.VALUE.iloc[-1] - 100)
val19 = df.loc['2019', 'VALUE'].mean()
lt3m = df.VALUE.iloc[-3:].mean()
url = 'https://www.bis.org/statistics/eer.htm'
text = (f'The Bank for International Settlements (BIS) \href{{{url}}}{{calculates}} '+
        '\\textbf{real effective exchange rates} for many countries, on a monthly '+
        f'basis. As of {ltdt}, the US dollar real effective exchange rate has '+
        f'{ltval} since 2010. In 2019, the index average was {val19:.1f}. Over the '+
        f'past three months, the index average value was {lt3m:.1f}.')
write_txt(text_dir / 'reer.txt', text)
print(text)
node = end_node(df.VALUE, 'red', date='m', full_year=True, offset=-0.35)
write_txt(text_dir / 'reer_node.txt', node)

The Bank for International Settlements (BIS) \href{https://www.bis.org/statistics/eer.htm}{calculates} \textbf{real effective exchange rates} for many countries, on a monthly basis. As of September 2022, the US dollar real effective exchange rate has increased 32.1 percent since 2010. In 2019, the index average was 117.1. Over the past three months, the index average value was 130.3.


### Decomposition of US Public Debt Dynamics

In [21]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=005f835d2ffea228372c2cc838173fa5&lastobs=&'+
       'from=01/01/1988&to=12/31/2021&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
n = {'FA316006005.A': 'Fiscal balance',
     'FA316130001.A': 'Interest paid',
     'FL314104005.A': 'DSL',
     'FL314190005.A': 'Liab',
     'FA315000905.A': 'Borrowing'}
data = df[n.keys()].rename(n, axis=1)

In [22]:
data['Primary balance'] = data['Fiscal balance'] + data['Interest paid']
gdpa = (nipa_df(retrieve_table('T10105A')['Data'], ['A191RC'])
        ['A191RC'])
data['GDP'] = gdpa
data['pi_'] = (nipa_df(retrieve_table('T10104A')['Data'], ['A191RG'])
               ['A191RG'])
data['defl'] = data['pi_'].iloc[-1] / data['pi_']
data['Real GDP'] = data['defl'] * data['GDP']

pb = -data['Primary balance']
i = data['Interest paid'] / data['DSL'].shift()
pi = data['pi_'].pct_change()
g = data['Real GDP'].pct_change()
lmbda = data['GDP'].pct_change()
d = data['DSL'] / data['GDP']
p = pb / data['GDP']

res = (((i / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()) +
       p)

data['calc'] = res

g2 = 0
res3 = (((i / (1 + lmbda - g)) * d.shift()) - 
       ((pi / (1 + lmbda - g)) * d.shift()) - 
       ((g2 / (1 + g2)) * d.shift()) +
       p)

res4 = (((pi / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()) +
       p)

res5 = (((i / (1 + lmbda)) * d.shift()) - 
       ((pi / (1 + lmbda)) * d.shift()) - 
       ((g / (1 + g)) * d.shift()))

data['e_gdp'] = (res - res3)
data['e_p'] = (res - res5)
data['e_r'] = (res - res4)
data['calc2'] = data['e_gdp'] + data['e_p'] + data['e_r']
data['sfa'] = d.diff() - data['calc2']
data['total'] = d.diff()
final = data[['e_gdp', 'e_p', 'e_r', 'sfa', 'total']] * 100
final = final.dropna()
final.index = final.index + pd.DateOffset(months=6)
final.to_csv(data_dir / 'debt_dynamics.csv', index_label='date')

col = {'p': 'orange!60!yellow!86!white',
       'gdp': 'violet!80!white', 'r': 'green!80!yellow!68!black', 
       'sfa': 'blue!90!cyan!78!white'}

lt = final.iloc[-1]
ltdt = final.index[-1].year
totch = value_text(lt['total'], 'increase_by', 'pp', threshold=0.1)
pbch = value_text(lt['e_p'], 'contribution_to', 'pp', casual=True, threshold=0.1)
rch = value_text(lt['e_r'], 'contribution', 'pp', casual=True, threshold=0.1)
gdpch = value_text(lt['e_gdp'], 'contribution', 'pp', casual=True, threshold=0.1)
sfach = value_text(lt['sfa'], 'contribution', 'pp', casual=True, threshold=0.1)
cmb = lt[['e_gdp', 'e_p', 'e_r']].sum()
cmbtxt = 'greater' if cmb > d.diff().iloc[-1] else 'less'
text = (f'In {ltdt}, the debt to GDP ratio {totch} (see '+
        '\\begin{tikzpicture} \\node[line width=0.3mm, circle, '+
        'draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; '+
        '\end{tikzpicture}). The primary balance '+
        f'{pbch} the debt to GDP ratio {c_box(col["p"])}, economic '+
        f'growth {gdpch} {c_box(col["gdp"])}, and real interest rates '+
        f'{rch} {c_box(col["r"])}. These combined factors were '+
        f'{cmbtxt} than the actual change in liabilities; '+
        'the adjustment to reconcile stocks and flows '+
        f'{sfach} {c_box(col["sfa"])}.')
write_txt(text_dir / 'debt_dynamics.txt', text)
print(text)

In 2021, the debt to GDP ratio decreased by 3.6 percentage points (see \begin{tikzpicture} \node[line width=0.3mm, circle, draw=black, scale=0.4, aspect=0.8] (d) at (0,0) {}; \end{tikzpicture}). The primary balance added 9.7 percentage points to the debt to GDP ratio (see \cbox{orange!60!yellow!86!white}), economic growth subtracted 6.2 percentage points (see \cbox{violet!80!white}), and real interest rates subtracted 2.3 percentage points (see \cbox{green!80!yellow!68!black}). These combined factors were greater than the actual change in liabilities; the adjustment to reconcile stocks and flows subtracted 4.9 percentage points (see \cbox{blue!90!cyan!78!white}).


### Government Expenditures by Type

Longer-term data from OMB Historical Data Table 8.3

Shorter-term data from Treasury Monthly Statement Table 9

In [23]:
# Retrieve long-term data
url = 'https://www.whitehouse.gov/wp-content/uploads/2021/05/hist08z3_fy22.xlsx'
data = pd.read_excel(url, header=[2,3,4], skipfooter=2, index_col=0)
data = data.rename(columns=lambda x: x if not 'Unnamed' in str(x) else '')
data.columns = [' '.join(col).strip() for col in data.columns.values]
data['Mandatory'] = data.loc[:,data.columns.duplicated()]
data = data.loc[:,~data.columns.duplicated()]
data = data.rename({'Total': 'Discretionary', '': 'Total'}, axis=1).loc['1989':]
data.columns = [i.replace('\n', ' ').replace('Mandatory Programmatic', 'MP') 
                for i in data.columns]
data.to_csv(data_dir / 'gov_exp_omb_raw.csv', index_label='date')

/home/brian/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [24]:
data = pd.read_csv(data_dir / 'gov_exp_omb_raw.csv', index_col='date')
res = data[['Net Interest', 'Non- defense', 'National Defense', 
            'MP Social Security', 'MP Medicare']].copy()
res['Social Safety Net'] = (data['MP Medicaid'] + 
                            data['MP Other Means  Tested Entitlements (1)'])
res['Other'] = 100 - res.sum(axis=1)
res = res.loc[~res.index.str.contains('estimate')]
res.index = pd.to_datetime(res.index) + pd.DateOffset(months=6)
res.to_csv(data_dir / 'gov_exp_omb.csv', index_label='date')

cols = ['National Defense', 'Non- defense', 'MP Medicare', 'MP Social Security', 
        'Social Safety Net', 'Other', 'Net Interest']
sdf = res[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 4)).to_dict()
val = sdf.to_dict()
dt = dtxt(res.index[-1] + pd.DateOffset(weeks=2))['datetime']
nodes = [f'\\absnode{{{dt}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}\%}}' for i in cols]
dt2 = dtxt(final.index[-1] - pd.DateOffset(months=24))['datetime']
dv = dtxt(res.index[-1])['year']
dtnode = f'\\absnode{{{dt2}}}{{{108}}}{{\\footnotesize {dv}}}'
nodetext = '\n'.join([dtnode] + nodes)
write_txt(text_dir / 'gov_exp_nodes.txt', nodetext)

val19 = res.loc['2019-07-01']
val89 = res.loc['1989-07-01']
def19 = val19['National Defense']
def89 = val89['National Defense']
ni19 = val19['Net Interest']
ni89 = val89['Net Interest']
combcat = ['MP Medicare', 'MP Social Security', 'Social Safety Net']
comb19 = val19.loc[combcat].sum()
comb89 = val89.loc[combcat].sum()
colors = {'Defense': 'green!80!blue!68!black',
          'NonDef': 'green!78!black',
          'NI': 'orange!80!yellow',
          'SS': 'cyan!50!blue!95!white',
          'Medicare': 'blue!65!black',
          'SSN': 'cyan!95!white'}

text = (f'national defense spending fell to {def19:.1f} percent '+
        f'of outlays in 2019 from {def89:.1f} percent in 1989 '+
        f'{c_box(colors["Defense"])}. Discretionary non-defense '+
        'spending maintained a relative stable share of spending '+
        f'over the period {c_box(colors["NonDef"])}. Net interest '+
        'expense, the cost of federal borrowing, fell along with '+
        f'long-term interest rates, to {ni19:.1f} percent of outlays '+
        f'in 2019 from {ni89:.1f} percent in 1989 '+
        f'{c_box(colors["NI"])}. \n\n Offsetting the reduction in '+
        'spending on interest and national defense, Medicare and '+
        'Social Security now make up a larger share of federal '+
        'spending, as a larger share of people are retirement age. '+
        'Likewise, spending on the social safety net (means-tested '+
        'benefits and Medicaid) increased as employment-to-population '+
        'ratios fell and Medicaid was expanded. Medicare '+
        f'{c_box(colors["Medicare"])}, Social Security '+
        f'{c_box(colors["SS"])}, and the social safety net '+
        f'{c_box(colors["SSN"])} combine to comprise {comb19:.1f} '+
        'percent of federal spending in 2019, compared to '+
        f'{comb89:.1f} percent in 1989. ')
write_txt(text_dir / 'gov_exp_comp1.txt', text)
print(text)

national defense spending fell to 15.2 percent of outlays in 2019 from 26.6 percent in 1989 (see \cbox{green!80!blue!68!black}). Discretionary non-defense spending maintained a relative stable share of spending over the period (see \cbox{green!78!black}). Net interest expense, the cost of federal borrowing, fell along with long-term interest rates, to 8.4 percent of outlays in 2019 from 14.8 percent in 1989 (see \cbox{orange!80!yellow}). 

 Offsetting the reduction in spending on interest and national defense, Medicare and Social Security now make up a larger share of federal spending, as a larger share of people are retirement age. Likewise, spending on the social safety net (means-tested benefits and Medicaid) increased as employment-to-population ratios fell and Medicaid was expanded. Medicare (see \cbox{blue!65!black}), Social Security (see \cbox{cyan!50!blue!95!white}), and the social safety net (see \cbox{cyan!95!white}) combine to comprise 54.8 percent of federal spending in 201

In [25]:
#Retrieve short-term data
cols = ['record_date', 'classification_desc', 
        'current_month_rcpt_outly_amt']
fields = ','.join(cols)
lines = ','.join([str(i) for i in list(range(14, 34))])
url = ('https://api.fiscaldata.treasury.gov/services/api/'+
       'fiscal_service/v1/accounting/mts/mts_table_9'+
       f'?filter=src_line_nbr:in:({lines})&format=csv&'+
       f'fields={fields}&page[size]=6000')
df = (pd.read_csv(url, index_col=cols[:2], parse_dates=True)
        ['current_month_rcpt_outly_amt']
        .unstack().astype('float').divide(1_000_000))
df.to_csv(data_dir / 'gov_exp_mts_raw.csv', index_label='date')

In [26]:
df = pd.read_csv(data_dir / 'gov_exp_mts_raw.csv', index_col='date', 
                 parse_dates=True)
oth = ['Administration of Justice', 'Agriculture',
       'Commerce and Housing Credit', 
       'Community and Regional Development',
       'Education, Training, Employment, and Social Services', 
       'Energy', 'General Government', 
       'General Science, Space, and Technology',
       'International Affairs', 'Natural Resources and Environment', 
       'Transportation', 'Undistributed Offsetting Receipts', 
       'Veterans Benefits and Services']
df['Other'] = df[oth].sum(axis=1)
res = df[['Income Security', 'Social Security', 'Health', 
          'National Defense', 'Medicare', 'Net Interest', 'Other']].copy()
final = ((res.divide(res.sum(axis=1), axis=0) * 100)
             .rolling(12).mean().dropna())
final.loc['2016-12-01':].to_csv(data_dir / 'gov_exp_mts.csv', 
                                index_label='date')

cols = ['National Defense', 'Health', 'Medicare', 'Social Security', 
        'Income Security', 'Other', 'Net Interest']
sdf = final[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 4)).to_dict()
val = sdf.to_dict()
dt = dtxt(final.index[-1] + pd.DateOffset(weeks=1))['datetime']
nodes = [f'\\absnode{{{dt}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}\%}}' 
         for i in cols]
dt2 = dtxt(final.index[-1] - pd.DateOffset(months=3))['datetime']
dv = dtxt(final.index[-1])['mon6']
dtnode = f'\\absnode{{{dt2}}}{{{108}}}{{\\footnotesize {dv}}}'
nodetext = '\n'.join([dtnode] + nodes)
write_txt(text_dir / 'gov_exp_nodes2.txt', nodetext)

ltdt = dtxt(final.index[-1])['mon1']
lt = final.iloc[-1]
incseclt = lt['Income Security']
maxdt = dtxt(final[['Income Security', 'Other']].sum(axis=1).idxmax())
maxmon = maxdt['mon1']
mx = final.loc[maxdt['datetime']]
pc = final.loc['2019-12-31']
incsecmx = mx['Income Security']
incsecpc = pc['Income Security']
text = ('Income security, which includes economic impact payments, the '+
        'child tax credit, unemployment compensation, food and nutrition '+
        'assistance, federal employee retirement and disability, and '+
        f'housing assistance, was {incseclt:.1f} percent of federal spending '+
        f'over the 12 months ending {ltdt} {c_box("brown")}. At its peak, '+
        f'over the 12 months ending {maxmon}, income security comprised '+
        f'{incsecmx:.1f} percent of federal spending. Pre-pandemic, in 2019, '+
        f'the category comprised {incsecpc:.1f} percent. ')
write_txt(text_dir / 'gov_exp_comp2.txt', text)
print(text)

cols2 = {'Income Security': 'brown', 'Health': 'violet!50!black', 
         'Medicare': 'blue!65!black', 
         'Social Security': 'cyan!50!blue!95!white', 
         'National Defense': 'green!80!blue!68!black',  
         'Net Interest': 'orange!80!yellow', 
         'Other: ': 'yellow!50!brown!55!white'}
col_names = {k: f'\hspace{{-1mm}}\cbox{{{v}}}{k}' for k, v in cols2.items()}
col_names2 = {i: f'\hspace{{6mm}}{i}' for i in oth}

data = df.rolling(12).sum()
data = data.divide(data.Total, axis=0) * 100
data = data.rename({'Other': 'Other: '}, axis=1)
data = data[list(cols2.keys()) + oth]
tbl = data.iloc[-3:].iloc[::-1].T
dmax = data.loc[maxdt['datetime']]
d19 = data.loc['2019'].mean().rename('2019')
d17 = data.loc['2017'].mean().rename('2017')
tbl = tbl.join(dmax)
tbl.columns = [dtxt(i)['mon2'] for i in tbl.columns]
tbl = tbl.join(d19).join(d17)
tbl = tbl.rename(col_names).rename(col_names2)
tbl.index = tbl.index.str.replace('and', '\&')
tbl = tbl.round(1).applymap('{:.1f}'.format)
tbl = tbl.rename({'\hspace{6mm}Education, Training, Employment, \& Social Services': 
            '\hspace{6mm}Educ., Training, Employment, \& Social Serv.'})
(tbl.to_csv(data_dir / 'gov_exp_comp.tex', sep='&', 
            lineterminator='\\\ ', quotechar=' '))

ltval = data['Other: '].iloc[-1]
maxval = data.loc[maxdt['datetime'], 'Other: ']
val19 = data.loc['2019', 'Other: '].mean()
text = (f"The category labeled ``other'' in the above-right chart "+
        f'includes several subcategories worth examining. The '+
        f'category increased to {ltval:.1f} percent of federal '+
        f'spending during the 12 months ending {ltdt}, from '+
        f'{maxval:.1f} percent during the 12 months ending {maxmon} '+
        f'{c_box(cols2["Other: "])}. Prior to the pandemic, in 2019, '+
        f'the category was {val19:.1f} percent of spending. ')
write_txt(text_dir / 'gov_exp_comp3.txt', text)
print('\n', text)

Income security, which includes economic impact payments, the child tax credit, unemployment compensation, food and nutrition assistance, federal employee retirement and disability, and housing assistance, was 13.9 percent of federal spending over the 12 months ending October 2022 (see \cbox{brown}). At its peak, over the 12 months ending March 2021, income security comprised 24.7 percent of federal spending. Pre-pandemic, in 2019, the category comprised 11.3 percent. 

 The category labeled ``other'' in the above-right chart includes several subcategories worth examining. The category increased to 20.3 percent of federal spending during the 12 months ending October 2022, from 24.2 percent during the 12 months ending March 2021 (see \cbox{yellow!50!brown!55!white}). Prior to the pandemic, in 2019, the category was 12.8 percent of spending. 
